# Analyze Unlabel from Re-label 2

1. Import data from Relabel_result_2.xlsx
2. Select data for each observation that are unlabeled
3. Find patterns using RegEx and N-grams (use function from Analyze_group_c.ipynb)
4. Find word before and after keyword
5. Add the result to negative, positive and uncertain key in Relabel_group_c.ipynb
6. Re-label again in Relabel_group_c.ipynb

## Import Data

In [861]:
# from google.colab import drive

# drive.mount('/content/drive')

In [862]:
# %cd /content/drive/My Drive/Pre-WiL

In [863]:
import pandas as pd

cardio_data_old = pd.read_excel('Relabel_result.xlsx', sheet_name='Cardiomegaly')
pleural_data_old = pd.read_excel('Relabel_result.xlsx', sheet_name='Pleural Effusion')
atelec_data_old = pd.read_excel('Relabel_result.xlsx', sheet_name='Atelectasis')
lesion_data_old = pd.read_excel('Relabel_result.xlsx', sheet_name='Lung Lesion')
v1_data_old = pd.read_excel('Relabel_result.xlsx', sheet_name='Inspectra Lung Opacity v1')
edema_data_old = pd.read_excel('Relabel_result.xlsx', sheet_name='Edema')

In [864]:
import pandas as pd

cardio_data = pd.read_excel('Relabel_result_2.xlsx', sheet_name='Cardiomegaly')
pleural_data = pd.read_excel('Relabel_result_2.xlsx', sheet_name='Pleural Effusion')
atelec_data = pd.read_excel('Relabel_result_2.xlsx', sheet_name='Atelectasis')
lesion_data = pd.read_excel('Relabel_result_2.xlsx', sheet_name='Lung Lesion')
v1_data = pd.read_excel('Relabel_result_2.xlsx', sheet_name='Inspectra Lung Opacity v1')
edema_data = pd.read_excel('Relabel_result_2.xlsx', sheet_name='Edema')

## Function

In [865]:
# create dataframe of each observations
def selectData(df, name, value):
  # choose rows
  if value == None:
    name_df = df[df[name + ' Ground Truth'].isna()]
  else:
    name_df = df[df[name + ' Ground Truth'] == value]

  return name_df

In [866]:
# create list of reports
import re
from ast import literal_eval

def createReportList(df):
  return list(map(literal_eval, df['Report List']))

In [867]:
# create n-grams
from collections import defaultdict
import re
from nltk import ngrams

def createPatternNgrams(data, n, keyword=None):
  pattern_dict = defaultdict(lambda: 0)
  for report in data:
    for sent in report[1:]:
      if (keyword != None and re.search(keyword, sent)) or keyword == None:
        for word in ngrams(sent.split(), n):
          pattern_dict[word] += 1    

  pattern_dict = sorted(pattern_dict.items(), key=lambda x: x[1], reverse=True)

  return dict(pattern_dict)

In [868]:
# find word before input word
def findWordBefore(bigrams_dict, word):
  word_before = defaultdict(lambda: 0)
  for k,v in bigrams_dict.items():
    if re.search(word, k[1]):
      word_before[k[0]] += v

  word_before = sorted(word_before.items(), key=lambda x: x[1], reverse=True)

  return dict(word_before)

def findWordBefore2(trigrams_dict, word):
  word_before = defaultdict(lambda: 0)
  for k,v in trigrams_dict.items():
    if re.search(word, k[2]):
      word_before[k[0]] += v

  word_before = sorted(word_before.items(), key=lambda x: x[1], reverse=True)

  return dict(word_before)

In [869]:
# find word after v.to be
def findWordAfter(trigrams_dict, word):
  word_after = defaultdict(lambda: 0)
  for k,v in trigrams_dict.items():
    if re.search(word, k[0]) and re.search(r'is|are|was|were', k[1]):
      word_after[k[2]] += v

  word_after = sorted(word_after.items(), key=lambda x: x[1], reverse=True)

  return dict(word_after)

In [870]:
def findDiffData(name, oldDf, df):
    for i in df.index:
        if not (pd.isna(df.at[i, name+' Ground Truth']) and  pd.isna(oldDf.at[i, name+' Ground Truth'])):
            if df.at[i, name+' Ground Truth'] != oldDf.at[i, name+' Ground Truth']:
                print(oldDf.at[i, name+' Ground Truth'], df.at[i, name+' Ground Truth'], df.at[i,'Report List'])

# Cardiomegaly

In [871]:
# define keyword
cardio_keyword = r'card|heart|lv|ventricular enlarge'

## Ground truth 0

Prepare Data

In [872]:
# choose reports
cardio_df_0 = selectData(cardio_data, 'Cardiomegaly', 0.0)
cardio_df_0

,Reports,Report List,Cardiomegaly BERT Labeler,Cardiomegaly Inspectra Labeler,Cardiomegaly Ground Truth,Cardiomegaly Key Detect
1,CXR PA upright\n The chest shows round calcif...,"['<s> cxr pa upright <\\s>', '<s> the chest sh...",0,1,0.0,"<re.Match object; span=(12, 25), match='heart ..."
10,CHEST\n\nMinimal fibrotic change at LUL.\nThe ...,"['<s> chest <\\s>', '<s> minimal fibrotic chan...",0,1,0.0,0.5
14,CHEST.;\nLarge soft tissue mass at right parat...,"['<s> chest <\\s>', '<s> large soft tissue mas...",1,0,0.0,"<re.Match object; span=(8, 62), match='no card..."
16,CHEST PA UPRIGHT\n\nhistory of ca lung post ri...,"['<s> chest pa upright <\\s>', '<s> history of...",1,0,0.0,"<re.Match object; span=(8, 38), match='no card..."
19,CXR \n\nHistory of hemoptysis\n\nThere is mass...,"['<s> cxr <\\s>', '<s> history of hemoptysis <...",1,0,0.0,"<re.Match object; span=(17, 42), match='no car..."
...,...,...,...,...,...,...
39236,CXR PA UPRIGHT\n\nFINDINGS : No detectable pu...,"['<s> cxr pa upright <\\s>', '<s> findings no ...",1,0,0.0,"<re.Match object; span=(8, 41), match='no card..."
39237,CXR PA UPRIGHT\n\nFINDINGS : No detectable pu...,"['<s> cxr pa upright <\\s>', '<s> findings no ...",1,0,0.0,"<re.Match object; span=(8, 41), match='no card..."
39238,CXR(PA)\n\n IMP:\n No active pulmonary infi...,"['<s> cxr pa <\\s>', '<s> imp <\\s>', '<s> no ...",1,0,0.0,"<re.Match object; span=(17, 42), match='no car..."
39239,CXR(PA)\n\n IMP:\n No active pulmonary infi...,"['<s> cxr pa <\\s>', '<s> imp <\\s>', '<s> no ...",1,0,0.0,"<re.Match object; span=(17, 42), match='no car..."


In [873]:
# create report list
cardio_report_0 = createReportList(cardio_df_0)
cardio_report_0[0]

['<s> cxr pa upright <\\s>',
 '<s> the chest shows round calcification at cardiac shadow should be calcification at left lower lung <\\s>',
 '<s> the right lung is clear <\\s>',
 '<s> the heart is not enlarged <\\s>',
 '<s> both costophrenic are clear <\\s>',
 '<s> the bony thorax is intact <\\s>']

Find N-grams Pattern

In [874]:
cardio_bigram_0 = createPatternNgrams(cardio_report_0, 2, cardio_keyword)
list(cardio_bigram_0.items())[:20]

[(('no', 'cardiomegaly'), 30027),
 (('<s>', 'no'), 29850),
 (('cardiomegaly', 'is'), 27439),
 (('is', 'observed'), 10758),
 (('observed', '<\\s>'), 10752),
 (('seen', '<\\s>'), 9984),
 (('is', 'seen'), 9956),
 (('noted', '<\\s>'), 3450),
 (('is', 'noted'), 3402),
 (('found', '<\\s>'), 3091),
 (('is', 'found'), 3089),
 (('cardiomegaly', '<\\s>'), 2420),
 (('<s>', 'the'), 796),
 (('cardiothoracic', 'ratio'), 675),
 (('the', 'heart'), 583),
 (('<s>', 'there'), 568),
 (('there', 'is'), 551),
 (('ratio', 'is'), 540),
 (('is', 'no'), 532),
 (('heart', 'is'), 528)]

In [875]:
cardio_trigram_0 = createPatternNgrams(cardio_report_0, 3, cardio_keyword)
list(cardio_trigram_0.items())[:20]

[(('<s>', 'no', 'cardiomegaly'), 29516),
 (('no', 'cardiomegaly', 'is'), 27370),
 (('is', 'observed', '<\\s>'), 10743),
 (('cardiomegaly', 'is', 'observed'), 10726),
 (('is', 'seen', '<\\s>'), 9928),
 (('cardiomegaly', 'is', 'seen'), 9803),
 (('is', 'noted', '<\\s>'), 3379),
 (('cardiomegaly', 'is', 'noted'), 3276),
 (('is', 'found', '<\\s>'), 3089),
 (('cardiomegaly', 'is', 'found'), 3086),
 (('no', 'cardiomegaly', '<\\s>'), 2206),
 (('<s>', 'the', 'heart'), 567),
 (('<s>', 'there', 'is'), 546),
 (('cardiothoracic', 'ratio', 'is'), 535),
 (('there', 'is', 'no'), 516),
 (('is', 'no', 'cardiomegaly'), 494),
 (('ratio', 'is', 'about'), 465),
 (('about', '0.5', '<\\s>'), 451),
 (('is', 'about', '0.5'), 446),
 (('heart', 'is', 'not'), 400)]

## Ground truth 1

Prepare Data

In [876]:
# choose reports
cardio_df_1 = selectData(cardio_data, 'Cardiomegaly', 1.0)
cardio_df_1

,Reports,Report List,Cardiomegaly BERT Labeler,Cardiomegaly Inspectra Labeler,Cardiomegaly Ground Truth,Cardiomegaly Key Detect
0,CHEST :\nP.A. upright view .\nNo active pulmon...,"['<s> chest <\\s>', '<s> p.a <\\s>', '<s> upri...",1,0,1.0,"<re.Match object; span=(8, 33), match='lvh is ..."
5,CHEST\n\nNo definite pulmonary infiltration.\n...,"['<s> chest <\\s>', '<s> no definite pulmonary...",0,1,1.0,0.51
8,Chest;\n\n Left ventricular enlargement with ...,"['<s> chest <\\s>', '<s> left ventricular enla...",1,0,1.0,"<re.Match object; span=(8, 32), match='left ve..."
9,CHEST\n\nFibrotic change at left basal lung.\n...,"['<s> chest <\\s>', '<s> fibrotic change at le...",0,1,1.0,0.54
11,CHEST\n\nNo definite pulmonary infiltration.\n...,"['<s> chest <\\s>', '<s> no definite pulmonary...",0,1,1.0,0.54
...,...,...,...,...,...,...
39218,CHEST FILM\n\nNo pulmonary infiltration is see...,"['<s> chest film <\\s>', '<s> no pulmonary inf...",1,0,1.0,"<re.Match object; span=(8, 28), match='cardiom..."
39224,CHEST FILM\n\nNo pulmonary infiltration is see...,"['<s> chest film <\\s>', '<s> no pulmonary inf...",1,0,1.0,"<re.Match object; span=(8, 28), match='cardiom..."
39231,Chest PA upright\n \n No pulmonary infiltrat...,"['<s> chest pa upright <\\s>', '<s> no pulmona...",1,0,1.0,"<re.Match object; span=(8, 40), match='promine..."
39232,CXR PA \n\nNo definite pulmonary infiltration ...,"['<s> cxr pa <\\s>', '<s> no definite pulmonar...",0,1,1.0,0.51


In [877]:
# create report list
cardio_report_1 = createReportList(cardio_df_1)
cardio_report_1[0]

['<s> chest <\\s>',
 '<s> p.a <\\s>',
 '<s> upright view <\\s>',
 '<s> no active pulmonary disease is detected <\\s>',
 '<s> lvh is observed <\\s>',
 '<s> calcified and tortuous aortic arch are noted <\\s>']

Find N-grams Pattern

In [878]:
cardio_bigram_1 = createPatternNgrams(cardio_report_1, 2, cardio_keyword)
list(cardio_bigram_1.items())[:20]

[(('<s>', 'cardiomegaly'), 1381),
 (('noted', '<\\s>'), 1228),
 (('is', 'noted'), 1220),
 (('cardiomegaly', 'is'), 1201),
 (('cardiothoracic', 'ratio'), 973),
 (('<s>', 'prominent'), 887),
 (('ratio', 'is'), 444),
 (('ratio', '<\\s>'), 436),
 (('is', 'about'), 326),
 (('prominent', 'cardiac'), 319),
 (('prominent', 'cardiothoracic'), 306),
 (('size', '<\\s>'), 294),
 (('cardiac', 'shadow'), 289),
 (('prominent', 'heart'), 269),
 (('heart', 'size'), 246),
 (('shadow', '<\\s>'), 222),
 (('<s>', 'cardiothoracic'), 207),
 (('increased', 'cardiothoracic'), 201),
 (('<s>', 'increased'), 182),
 (('<s>', 'the'), 179)]

In [879]:
cardio_trigram_1 = createPatternNgrams(cardio_report_1, 3, cardio_keyword)
list(cardio_trigram_1.items())[:20]

[(('is', 'noted', '<\\s>'), 1213),
 (('<s>', 'cardiomegaly', 'is'), 1191),
 (('cardiomegaly', 'is', 'noted'), 1130),
 (('cardiothoracic', 'ratio', '<\\s>'), 435),
 (('cardiothoracic', 'ratio', 'is'), 416),
 (('ratio', 'is', 'about'), 326),
 (('<s>', 'prominent', 'cardiac'), 318),
 (('prominent', 'cardiothoracic', 'ratio'), 306),
 (('<s>', 'prominent', 'cardiothoracic'), 301),
 (('<s>', 'prominent', 'heart'), 237),
 (('prominent', 'heart', 'size'), 232),
 (('<s>', 'cardiothoracic', 'ratio'), 205),
 (('increased', 'cardiothoracic', 'ratio'), 200),
 (('cardiac', 'shadow', '<\\s>'), 193),
 (('prominent', 'cardiac', 'shadow'), 172),
 (('<s>', 'increased', 'cardiothoracic'), 171),
 (('heart', 'size', '<\\s>'), 160),
 (('<s>', 'the', 'cardiothoracic'), 156),
 (('the', 'cardiothoracic', 'ratio'), 156),
 (('about', '0.51', '<\\s>'), 149)]

## Ground truth -1

Prepare Data

In [880]:
# choose reports
cardio_df_u = selectData(cardio_data, 'Cardiomegaly', -1.0)
cardio_df_u

,Reports,Report List,Cardiomegaly BERT Labeler,Cardiomegaly Inspectra Labeler,Cardiomegaly Ground Truth,Cardiomegaly Key Detect
2,CXR PA upright\nNo demonstrable active pulmona...,"['<s> cxr pa upright <\\s>', '<s> no demonstra...",0,1,-1.0,"<re.Match object; span=(8, 41), match='borderl..."
3,CXR \n\nNo clinical history is provided.\n\nNo...,"['<s> cxr <\\s>', '<s> no clinical history is ...",0,1,-1.0,"<re.Match object; span=(17, 52), match='no cha..."
6,Chest PA upright;\n Rotate chest position.\n M...,"['<s> chest pa upright <\\s>', '<s> rotate che...",0,1,-1.0,"<re.Match object; span=(41, 61), match='mild p..."
17,CXR (PA upright)\n\nNo definite pulmonary inf...,"['<s> cxr pa upright <\\s>', '<s> no definite ...",1,0,-1.0,"<re.Match object; span=(8, 34), match='mild ca..."
21,Chest film\n\nReticulonodular infiltration bot...,"['<s> chest film <\\s>', '<s> reticulonodular ...",0,1,-1.0,"<re.Match object; span=(8, 41), match='borderl..."
...,...,...,...,...,...,...
39190,Chest:PA\n\nNo active pulmonary disease.\nDila...,"['<s> chest pa <\\s>', '<s> no active pulmonar...",0,1,-1.0,"<re.Match object; span=(8, 53), match='mild in..."
39200,"CXR\n\n Compare to CXR on 12/3/2018, markedly...","['<s> cxr <\\s>', '<s> compare to cxr on 12 3 ...",0,1,-1.0,"<re.Match object; span=(8, 73), match='borderl..."
39208,CXR PA upright\n\nHistory : --\n\nComparison s...,"['<s> cxr pa upright <\\s>', '<s> history <\\s...",1,0,-1.0,"<re.Match object; span=(8, 85), match='no chan..."
39209,"CXR PA upright\n\nHistory : DM,HT \n\nCompari...","['<s> cxr pa upright <\\s>', '<s> history dm h...",1,0,-1.0,"<re.Match object; span=(8, 49), match='no chan..."


In [881]:
# create report list
cardio_report_u = createReportList(cardio_df_u)
cardio_report_u[0]

['<s> cxr pa upright <\\s>',
 '<s> no demonstrable active pulmonary infiltration <\\s>',
 '<s> borderline cardiac size <\\s>',
 '<s> bony thorax is intact <\\s>',
 '<s> both costophrenic sulci are not remarkable <\\s>',
 '<s> imp no active pulmonary disease <\\s>']

Find N-grams Pattern

In [882]:
cardio_bigram_u = createPatternNgrams(cardio_report_u, 2, cardio_keyword)
list(cardio_bigram_u.items())[:20]

[(('<s>', 'mild'), 1410),
 (('mild', 'cardiomegaly'), 580),
 (('<s>', 'borderline'), 506),
 (('mild', 'prominent'), 506),
 (('prominent', 'heart'), 433),
 (('<s>', 'no'), 433),
 (('cardiomegaly', '<\\s>'), 359),
 (('size', '<\\s>'), 349),
 (('heart', 'size'), 322),
 (('cardiothoracic', 'ratio'), 321),
 (('aorta', '<\\s>'), 316),
 (('changed', 'of'), 307),
 (('no', 'changed'), 301),
 (('cardiac', 'size'), 255),
 (('vasculature', '<\\s>'), 240),
 (('of', 'cardiomegaly'), 236),
 (('pulmonary', 'vasculature'), 235),
 (('change', 'of'), 225),
 (('normal', 'pulmonary'), 224),
 (('borderline', 'cardiac'), 211)]

In [883]:
cardio_trigram_u = createPatternNgrams(cardio_report_u, 3, cardio_keyword)
list(cardio_trigram_u.items())[:20]

[(('<s>', 'mild', 'cardiomegaly'), 507),
 (('mild', 'prominent', 'heart'), 400),
 (('<s>', 'mild', 'prominent'), 400),
 (('prominent', 'heart', 'size'), 303),
 (('no', 'changed', 'of'), 297),
 (('<s>', 'no', 'changed'), 289),
 (('pulmonary', 'vasculature', '<\\s>'), 232),
 (('normal', 'pulmonary', 'vasculature'), 222),
 (('heart', 'size', '<\\s>'), 212),
 (('<s>', 'borderline', 'cardiac'), 211),
 (('borderline', 'cardiac', 'size'), 209),
 (('increased', 'cardiothoracic', 'ratio'), 201),
 (('mild', 'cardiomegaly', '<\\s>'), 195),
 (('no', 'change', 'of'), 176),
 (('mild', 'cardiomegaly', 'normal'), 165),
 (('cardiothoracic', 'ratio', '<\\s>'), 161),
 (('cardiomegaly', 'normal', 'pulmonary'), 158),
 (('<s>', 'mild', 'lvh'), 140),
 (('cardiac', 'size', 'with'), 138),
 (('changed', 'of', 'mild'), 133)]

## Unlabel

Prepare Data

In [884]:
# choose reports
cardio_df = selectData(cardio_data, 'Cardiomegaly', None)
cardio_df

,Reports,Report List,Cardiomegaly BERT Labeler,Cardiomegaly Inspectra Labeler,Cardiomegaly Ground Truth,Cardiomegaly Key Detect
4,Chest PA upright;\n R/O lung metastasis.\n\n E...,"['<s> chest pa upright <\\s>', '<s> r o lung m...",1,0,NaN,NaN
7,Chest PA uprihgt;\n Rt.lung destruction with f...,"['<s> chest pa uprihgt <\\s>', '<s> rt. lung d...",0,1,NaN,NaN
12,CXR PA upright\n The chest shows no definite ...,"['<s> cxr pa upright <\\s>', '<s> the chest sh...",0,1,NaN,NaN
15,CHEST PA UPRIGHT\n\nHistory: not available\nco...,"['<s> chest pa upright <\\s>', '<s> history no...",0,1,NaN,NaN
22,CHEST PA UPRIGHT\n\nHistory: not available\n\n...,"['<s> chest pa upright <\\s>', '<s> history no...",1,0,NaN,NaN
...,...,...,...,...,...,...
38828,CXR PA\n\nThere is right side aortic arch with...,"['<s> cxr pa <\\s>', '<s> there is right side ...",1,0,NaN,NaN
38882,CXR PA\n\nGlobular heart shape is noted.\nBlun...,"['<s> cxr pa <\\s>', '<s> globular heart shape...",0,1,NaN,NaN
39016,Chest:PA\nPost thoracic surgery with prostheti...,"['<s> chest pa <\\s>', '<s> post thoracic surg...",1,0,NaN,NaN
39175,CXR PA UPRIGHT\n\nInterstitial infiltration in...,"['<s> cxr pa upright <\\s>', '<s> interstitial...",0,1,NaN,NaN


In [885]:
# create report list
cardio_report = createReportList(cardio_df)
cardio_report[0]

['<s> chest pa upright <\\s>',
 '<s> r o lung metastasis <\\s>',
 '<s> enlarged size and increased number of diffuse pulmonary nodules in both lungs are seen <\\s>',
 '<s> much more rt.pleural effusion is also shown <\\s>',
 '<s> ds <\\s>']

Find N-grams Pattern

In [886]:
cardio_bigram = createPatternNgrams(cardio_report, 2)
list(cardio_bigram.items())[:20]

[(('<s>', 'no'), 852),
 (('pleural', 'effusion'), 443),
 (('bony', 'thorax'), 412),
 (('noted', '<\\s>'), 358),
 (('pulmonary', 'infiltration'), 353),
 (('seen', '<\\s>'), 343),
 (('intact', '<\\s>'), 339),
 (('clear', '<\\s>'), 338),
 (('<s>', 'bony'), 326),
 (('is', 'noted'), 324),
 (('is', 'seen'), 318),
 (('<s>', 'the'), 313),
 (('thorax', 'is'), 311),
 (('effusion', '<\\s>'), 281),
 (('are', 'clear'), 267),
 (('is', 'intact'), 263),
 (('<s>', 'both'), 251),
 (('both', 'costophrenic'), 251),
 (('no', 'active'), 225),
 (('costophrenic', 'angles'), 220)]

In [887]:
cardio_trigram = createPatternNgrams(cardio_report, 3)
list(cardio_trigram.items())[:20]

[(('bony', 'thorax', 'is'), 308),
 (('is', 'seen', '<\\s>'), 283),
 (('is', 'noted', '<\\s>'), 282),
 (('<s>', 'bony', 'thorax'), 269),
 (('are', 'clear', '<\\s>'), 266),
 (('is', 'intact', '<\\s>'), 262),
 (('pleural', 'effusion', '<\\s>'), 243),
 (('thorax', 'is', 'intact'), 230),
 (('<s>', 'both', 'costophrenic'), 222),
 (('costophrenic', 'angles', 'are'), 208),
 (('angles', 'are', 'clear'), 194),
 (('both', 'costophrenic', 'angles'), 185),
 (('no', 'pleural', 'effusion'), 184),
 (('<s>', 'no', 'pleural'), 174),
 (('no', 'active', 'pulmonary'), 166),
 (('<s>', 'no', 'active'), 160),
 (('<s>', 'there', 'is'), 153),
 (('active', 'pulmonary', 'infiltration'), 151),
 (('pulmonary', 'infiltration', 'or'), 140),
 (('pleural', 'effusion', 'is'), 138)]

In [888]:
cardio_fourgram = createPatternNgrams(cardio_report, 4)
list(cardio_fourgram.items())[:20]

[(('<s>', 'bony', 'thorax', 'is'), 235),
 (('thorax', 'is', 'intact', '<\\s>'), 229),
 (('bony', 'thorax', 'is', 'intact'), 227),
 (('angles', 'are', 'clear', '<\\s>'), 194),
 (('costophrenic', 'angles', 'are', 'clear'), 187),
 (('both', 'costophrenic', 'angles', 'are'), 176),
 (('<s>', 'no', 'pleural', 'effusion'), 174),
 (('<s>', 'both', 'costophrenic', 'angles'), 158),
 (('<s>', 'no', 'active', 'pulmonary'), 116),
 (('no', 'active', 'pulmonary', 'infiltration'), 104),
 (('effusion', 'is', 'seen', '<\\s>'), 97),
 (('no', 'pleural', 'effusion', '<\\s>'), 93),
 (('pulmonary', 'infiltration', 'or', 'nodule'), 87),
 (('no', 'pleural', 'effusion', 'is'), 84),
 (('<s>', 'no', 'pulmonary', 'infiltration'), 82),
 (('no', 'definite', 'pulmonary', 'infiltration'), 77),
 (('pleural', 'effusion', 'is', 'seen'), 73),
 (('active', 'pulmonary', 'infiltration', '<\\s>'), 73),
 (('bony', 'thorax', 'is', 'unremarkable'), 71),
 (('thorax', 'is', 'unremarkable', '<\\s>'), 71)]

In [889]:
cardio_bigramk = createPatternNgrams(cardio_report, 2, cardio_keyword)
list(cardio_bigramk.items())[:20]

[(('<s>', 'heart'), 112),
 (('<s>', 'the'), 69),
 (('heart', 'and'), 64),
 (('the', 'heart'), 51),
 (('heart', 'size'), 50),
 (('<s>', 'cardioemgaly'), 43),
 (('size', '<\\s>'), 42),
 (('and', 'mediastinum'), 38),
 (('to', 'the'), 37),
 (('aorta', '<\\s>'), 35),
 (('heart', 'is'), 33),
 (('cannot', 'be'), 31),
 (('cardiac', 'size'), 28),
 (('is', 'noted'), 27),
 (('cardioemgaly', 'with'), 26),
 (('the', 'left'), 26),
 (('of', 'heart'), 25),
 (('with', 'lve'), 24),
 (('<s>', 'there'), 24),
 (('there', 'is'), 24)]

In [890]:
cardio_trigramk = createPatternNgrams(cardio_report, 3, cardio_keyword)
list(cardio_trigramk.items())[:20]

[(('<s>', 'the', 'heart'), 49),
 (('<s>', 'heart', 'and'), 36),
 (('heart', 'and', 'mediastinum'), 34),
 (('<s>', 'heart', 'size'), 28),
 (('heart', 'and', 'great'), 22),
 (('<s>', 'cardioemgaly', 'with'), 22),
 (('with', 'lve', '<\\s>'), 22),
 (('<s>', 'there', 'is'), 22),
 (('the', 'heart', 'and'), 21),
 (('and', 'great', 'vessels'), 21),
 (('size', 'cannot', 'be'), 21),
 (('<s>', 'cardioemgaly', '<\\s>'), 21),
 (('to', 'the', 'left'), 19),
 (('heart', 'size', 'cannot'), 18),
 (('is', 'noted', '<\\s>'), 18),
 (('cannot', 'be', 'well'), 18),
 (('and', 'mediastinum', 'are'), 17),
 (('to', 'the', 'right'), 17),
 (('is', 'seen', '<\\s>'), 16),
 (('be', 'well', 'evaluated'), 15)]

In [891]:
cardio_fourgramk = createPatternNgrams(cardio_report, 4, cardio_keyword)
list(cardio_fourgramk.items())[:20]

[(('<s>', 'the', 'heart', 'and'), 21),
 (('heart', 'and', 'great', 'vessels'), 21),
 (('<s>', 'heart', 'and', 'mediastinum'), 21),
 (('heart', 'size', 'cannot', 'be'), 17),
 (('<s>', 'heart', 'size', 'cannot'), 15),
 (('heart', 'and', 'mediastinum', 'are'), 14),
 (('cannot', 'be', 'well', 'evaluated'), 14),
 (('and', 'great', 'vessels', 'are'), 12),
 (('the', 'heart', 'and', 'great'), 11),
 (('be', 'well', 'evaluated', '<\\s>'), 11),
 (('<s>', 'heart', 'and', 'great'), 11),
 (('size', 'cannot', 'be', 'well'), 11),
 (('heart', 'and', 'mediastinum', 'shift'), 10),
 (('and', 'mediastinum', 'shift', 'to'), 9),
 (('mediastinum', 'shift', 'to', 'the'), 9),
 (('shift', 'to', 'the', 'left'), 9),
 (('to', 'the', 'left', 'side'), 9),
 (('the', 'left', 'side', '<\\s>'), 9),
 (('<s>', 'cardimoegaly', 'with', 'lve'), 9),
 (('vessels', 'are', 'intact', '<\\s>'), 8)]

In [892]:
cardio_fivegramk = createPatternNgrams(cardio_report, 5, cardio_keyword)
list(cardio_fivegramk.items())[:20]

[(('<s>', 'heart', 'size', 'cannot', 'be'), 14),
 (('heart', 'and', 'great', 'vessels', 'are'), 12),
 (('<s>', 'the', 'heart', 'and', 'great'), 11),
 (('<s>', 'heart', 'and', 'great', 'vessels'), 11),
 (('the', 'heart', 'and', 'great', 'vessels'), 10),
 (('cannot', 'be', 'well', 'evaluated', '<\\s>'), 10),
 (('size', 'cannot', 'be', 'well', 'evaluated'), 10),
 (('<s>', 'heart', 'and', 'mediastinum', 'shift'), 9),
 (('heart', 'and', 'mediastinum', 'shift', 'to'), 9),
 (('and', 'mediastinum', 'shift', 'to', 'the'), 9),
 (('to', 'the', 'left', 'side', '<\\s>'), 9),
 (('<s>', 'heart', 'and', 'mediastinum', 'are'), 9),
 (('<s>', 'the', 'heart', 'and', 'mediastinum'), 8),
 (('<s>', 'cardimoegaly', 'with', 'lve', '<\\s>'), 8),
 (('and', 'great', 'vessels', 'are', 'intact'), 7),
 (('great', 'vessels', 'are', 'intact', '<\\s>'), 7),
 (('enarlged', 'cardaic', 'shadow', 'is', 'shown'), 7),
 (('cardaic', 'shadow', 'is', 'shown', '<\\s>'), 7),
 (('shift', 'to', 'the', 'left', 'side'), 7),
 (('heart

In [893]:
cardio_sixgramk = createPatternNgrams(cardio_report, 6, cardio_keyword)
list(cardio_sixgramk.items())[:20]

[(('<s>', 'the', 'heart', 'and', 'great', 'vessels'), 10),
 (('<s>', 'heart', 'and', 'mediastinum', 'shift', 'to'), 9),
 (('heart', 'and', 'mediastinum', 'shift', 'to', 'the'), 9),
 (('size', 'cannot', 'be', 'well', 'evaluated', '<\\s>'), 8),
 (('heart', 'and', 'great', 'vessels', 'are', 'intact'), 7),
 (('and', 'great', 'vessels', 'are', 'intact', '<\\s>'), 7),
 (('enarlged', 'cardaic', 'shadow', 'is', 'shown', '<\\s>'), 7),
 (('shift', 'to', 'the', 'left', 'side', '<\\s>'), 7),
 (('<s>', 'heart', 'and', 'mediastinum', 'are', 'unremarkable'), 7),
 (('heart', 'and', 'mediastinum', 'are', 'unremarkable', '<\\s>'), 7),
 (('<s>', 'heart', 'size', 'cannot', 'be', 'well'), 7),
 (('heart', 'size', 'cannot', 'be', 'well', 'evaluated'), 7),
 (('the', 'heart', 'and', 'great', 'vessels', 'are'), 6),
 (('<s>', 'heart', 'mediastinum', 'visualized', 'bony', 'thorax'), 6),
 (('<s>', 'heart', 'and', 'great', 'vessels', 'are'), 6),
 (('and', 'mediastinum', 'shift', 'to', 'the', 'left'), 6),
 (('<s>', 

## Difference between Result 1 and 2

In [894]:
findDiffData('Cardiomegaly', cardio_data_old, cardio_data)

nan 0.0 ['<s> cxr <\\s>', '<s> serial f u films showed persistent blunting of rt. costophrenic angle which might be rt. basal pleural thickening <\\s>', '<s> no pulmonary infiltration or cardiomegaly is noted <\\s>', '<s> otherwise is unremarkable <\\s>']
nan 0.0 ['<s> cxr <\\s>', '<s> bulging of rt. paramediastinal region and rt. hilum might be lymphadenopathy <\\s>', '<s> no definite pulmonary infiltration or cardiomegaly <\\s>', '<s> blunting of lt. costophrenic angle please exclude minimal pleural effusion <\\s>', '<s> old healed fracture rt. clavicle <\\s>']
nan 0.0 ['<s> chest <\\s>', '<s> minimal fibrotic change at both upper lobes with apical pleural thickening <\\s>', '<s> no significant cardiomegaly is noted <\\s>', '<s> no pleural effusion is seen <\\s>', '<s> bony thorax is intact <\\s>']
nan 0.0 ['<s> cxr <\\s>', '<s> calcified plaque or nodules at rul and lll <\\s>', '<s> fibrocalcific infiltration at lul is noted <\\s>', '<s> possible old pulmoanry tb <\\s>', '<s> no act

# Pleural Effusion

In [895]:
# define keyword
pleural_keyword = r'pleural|costophrenic'

## Ground truth 0

Prepare Data

In [896]:
# choose reports
pleural_df_0 = selectData(pleural_data, 'Pleural Effusion', 0.0)
pleural_df_0

,Reports,Report List,Pleural Effusion BERT Labeler,Pleural Effusion Inspectra Labeler,Pleural Effusion Ground Truth,Pleural Effusion Key Detect
7,CXR\nInterstitial infitration both lungs\nMild...,"['<s> cxr <\\s>', '<s> interstitial infitratio...",1,0,0.0,"<re.Match object; span=(0, 40), match='<s> <s>..."
8,Chest PA upright;\n compared to film on 20/4/2...,"['<s> chest pa upright <\\s>', '<s> compared t...",1,0,0.0,"<re.Match object; span=(0, 37), match='<s> <s>..."
11,CXR PA upright (13/7/2005)\n\n Fibros...,"['<s> cxr pa upright 13 7 2005 <\\s>', '<s> fi...",1,0,0.0,"<re.Match object; span=(0, 40), match='<s> <s>..."
18,Chest film\n\nMinimal fibrosis in RUL and inte...,"['<s> chest film <\\s>', '<s> minimal fibrosis...",1,0,0.0,"<re.Match object; span=(0, 40), match='<s> <s>..."
21,CXR PA upright\n By compared with previous ch...,"['<s> cxr pa upright <\\s>', '<s> by compared ...",1,0,0.0,"<re.Match object; span=(0, 52), match='<s> <s>..."
...,...,...,...,...,...,...
4544,CXR PA \n\nMinimal fibrosis at LLL.\nNormal ca...,"['<s> cxr pa <\\s>', '<s> minimal fibrosis at ...",1,0,0.0,"<re.Match object; span=(0, 40), match='<s> <s>..."
4547,CXR\n\nAs compared to prior film on 12 march 1...,"['<s> cxr <\\s>', '<s> as compared to prior fi...",0,1,0.0,"<re.Match object; span=(62, 106), match='no de..."
4551,CXR PA upright\n\n Compared to prior film on 3...,"['<s> cxr pa upright <\\s>', '<s> compared to ...",1,0,0.0,"<re.Match object; span=(8, 49), match='no defi..."
4558,CXR PA upright\n\nComparison to prior radiogra...,"['<s> cxr pa upright <\\s>', '<s> comparison t...",0,1,0.0,"<re.Match object; span=(8, 151), match='no int..."


In [897]:
# create report list
pleural_report_0 = createReportList(pleural_df_0)
pleural_report_0[0]

['<s> cxr <\\s>',
 '<s> interstitial infitration both lungs <\\s>',
 '<s> mild cardiomegaly <\\s>',
 '<s> both costophrenic sulci are not remarkable <\\s>']

Find N-grams Pattern

In [898]:
pleural_bigram_0 = createPatternNgrams(pleural_report_0, 2, pleural_keyword)
list(pleural_bigram_0.items())[:20]

[(('pleural', 'effusion'), 483),
 (('<s>', 'both'), 412),
 (('both', 'costophrenic'), 403),
 (('clear', '<\\s>'), 356),
 (('are', 'clear'), 316),
 (('effusion', '<\\s>'), 273),
 (('<s>', 'no'), 264),
 (('costophrenic', 'angles'), 230),
 (('angles', 'are'), 230),
 (('change', 'of'), 230),
 (('costophrenic', 'sulci'), 218),
 (('right', 'pleural'), 185),
 (('sulci', 'are'), 177),
 (('not', 'remarkable'), 165),
 (('remarkable', '<\\s>'), 165),
 (('are', 'not'), 164),
 (('no', 'interval'), 138),
 (('pleural', 'thickening'), 130),
 (('interval', 'change'), 129),
 (('left', 'pleural'), 128)]

In [899]:
pleural_trigram_0 = createPatternNgrams(pleural_report_0, 3, pleural_keyword)
list(pleural_trigram_0.items())[:20]

[(('<s>', 'both', 'costophrenic'), 399),
 (('are', 'clear', '<\\s>'), 316),
 (('pleural', 'effusion', '<\\s>'), 262),
 (('costophrenic', 'angles', 'are'), 230),
 (('angles', 'are', 'clear'), 227),
 (('both', 'costophrenic', 'angles'), 222),
 (('both', 'costophrenic', 'sulci'), 177),
 (('costophrenic', 'sulci', 'are'), 177),
 (('right', 'pleural', 'effusion'), 167),
 (('not', 'remarkable', '<\\s>'), 165),
 (('are', 'not', 'remarkable'), 164),
 (('sulci', 'are', 'not'), 129),
 (('no', 'interval', 'change'), 129),
 (('interval', 'change', 'of'), 129),
 (('of', 'right', 'pleural'), 120),
 (('<s>', 'no', 'interval'), 120),
 (('left', 'pleural', 'effusion'), 118),
 (('pleural', 'effusion', 'is'), 105),
 (('change', 'of', 'right'), 94),
 (('<s>', 'neither', 'pleural'), 72)]

## Ground truth 1

Prepare Data

In [900]:
# choose reports
pleural_df_1 = selectData(pleural_data, 'Pleural Effusion', 1.0)
pleural_df_1

,Reports,Report List,Pleural Effusion BERT Labeler,Pleural Effusion Inspectra Labeler,Pleural Effusion Ground Truth,Pleural Effusion Key Detect
4,CXR PA upright\n\nPulmonary infiltration at RL...,"['<s> cxr pa upright <\\s>', '<s> pulmonary in...",0,1,1.0,"<re.Match object; span=(42, 64), match='right ..."
9,CHEST PA UPRIGHT\n\nRight pleural effusion and...,"['<s> chest pa upright <\\s>', '<s> right pleu...",1,0,1.0,"<re.Match object; span=(8, 30), match='right p..."
10,CHEST PA UPRIGHT\n\nThere is righ tpleural eff...,"['<s> chest pa upright <\\s>', '<s> there is r...",1,0,1.0,"<re.Match object; span=(17, 39), match='righ t..."
17,CXR (PA_upright)\n\nFibrosis with reticulonod...,"['<s> cxr pa_upright <\\s>', '<s> fibrosis wit...",0,1,1.0,"<re.Match object; span=(8, 30), match='right p..."
25,CXR(PA)\n \nClinical history: loculated ple...,"['<s> cxr pa <\\s>', '<s> clinical history loc...",1,0,1.0,"<re.Match object; span=(0, 51), match='<s> <s>..."
...,...,...,...,...,...,...
4497,Chest PA upright \n\nHistory: Rt.pleural effus...,"['<s> chest pa upright <\\s>', '<s> history rt...",1,0,1.0,"<re.Match object; span=(0, 35), match='<s> <s>..."
4549,CXR PA upright\n\nLarge bleb at RUL\nOpacity a...,"['<s> cxr pa upright <\\s>', '<s> large bleb a...",0,1,1.0,"<re.Match object; span=(8, 30), match='right p..."
4555,Chest PA upright \n\nPrior noted bilateral ple...,"['<s> chest pa upright <\\s>', '<s> prior note...",1,0,1.0,"<re.Match object; span=(14, 46), match='noted ..."
4556,CHEST FILM\n\nReticulonodular infiltration at ...,"['<s> chest film <\\s>', '<s> reticulonodular ...",1,0,1.0,"<re.Match object; span=(24, 45), match='left p..."


In [901]:
# create report list
pleural_report_1 = createReportList(pleural_df_1)
pleural_report_1[0]

['<s> cxr pa upright <\\s>',
 '<s> pulmonary infiltration at rll and right pleural effusion <\\s>',
 '<s> normal cardiothoracic ratio <\\s>',
 '<s> bony thorax is intact <\\s>']

Find N-grams Pattern

In [902]:
pleural_bigram_1 = createPatternNgrams(pleural_report_1, 2, pleural_keyword)
list(pleural_bigram_1.items())[:20]

[(('pleural', 'effusion'), 347),
 (('effusion', '<\\s>'), 260),
 (('right', 'pleural'), 140),
 (('left', 'pleural'), 99),
 (('<s>', 'right'), 71),
 (('bilateral', 'pleural'), 59),
 (('effusion', 'is'), 58),
 (('<s>', 'left'), 56),
 (('<s>', 'bilateral'), 39),
 (('<s>', 'small'), 30),
 (('and', 'right'), 29),
 (('is', 'seen'), 29),
 (('costophrenic', 'angle'), 29),
 (('infiltration', 'at'), 28),
 (('seen', '<\\s>'), 28),
 (('noted', '<\\s>'), 27),
 (('<s>', 'no'), 27),
 (('is', 'noted'), 25),
 (('<s>', 'pulmonary'), 24),
 (('pulmonary', 'infiltration'), 24)]

In [903]:
pleural_trigram_1 = createPatternNgrams(pleural_report_1, 3, pleural_keyword)
list(pleural_trigram_1.items())[:20]

[(('pleural', 'effusion', '<\\s>'), 257),
 (('right', 'pleural', 'effusion'), 137),
 (('left', 'pleural', 'effusion'), 99),
 (('<s>', 'right', 'pleural'), 66),
 (('pleural', 'effusion', 'is'), 57),
 (('bilateral', 'pleural', 'effusion'), 50),
 (('<s>', 'left', 'pleural'), 48),
 (('<s>', 'bilateral', 'pleural'), 35),
 (('is', 'seen', '<\\s>'), 27),
 (('effusion', 'is', 'seen'), 26),
 (('and', 'right', 'pleural'), 25),
 (('<s>', 'pulmonary', 'infiltration'), 24),
 (('pulmonary', 'infiltration', 'at'), 23),
 (('is', 'noted', '<\\s>'), 23),
 (('infiltration', 'at', 'rll'), 21),
 (('costophrenic', 'angle', 'is'), 21),
 (('at', 'rll', 'and'), 19),
 (('rll', 'and', 'right'), 19),
 (('effusion', 'is', 'noted'), 17),
 (('left', 'costophrenic', 'angle'), 16)]

## Ground truth -1

Prepare Data

In [904]:
# choose reports
pleural_df_u = selectData(pleural_data, 'Pleural Effusion', -1.0)
pleural_df_u

,Reports,Report List,Pleural Effusion BERT Labeler,Pleural Effusion Inspectra Labeler,Pleural Effusion Ground Truth,Pleural Effusion Key Detect
0,"Chest PA upright;\n follow up film , compared ...","['<s> chest pa upright <\\s>', '<s> follow up ...",0,1,-1.0,"<re.Match object; span=(8, 53), match='no sign..."
1,Chest;\n\n Minimal fibrosis at RUL .\n The l...,"['<s> chest <\\s>', '<s> minimal fibrosis at r...",1,0,-1.0,"<re.Match object; span=(0, 51), match='<s> <s>..."
2,CHEST\n\nReticulonodular infiltration at RUL. ...,"['<s> chest <\\s>', '<s> reticulonodular infil...",0,1,-1.0,"<re.Match object; span=(8, 52), match='no sign..."
5,CXR\n\nHistory of CA lung\n\nAs compared to pr...,"['<s> cxr <\\s>', '<s> history of ca lung <\\s...",0,1,-1.0,"<re.Match object; span=(60, 95), match='no cha..."
12,CXR(PA)\n \n History: Pleural TB. Foll...,"['<s> cxr pa <\\s>', '<s> history pleural tb. ...",0,1,-1.0,"<re.Match object; span=(53, 104), match='no si..."
...,...,...,...,...,...,...
4548,"CXR\n\n Compare to CXR on 30/04/2017, no si...","['<s> cxr <\\s>', '<s> compare to cxr on 30 04...",0,1,-1.0,"<re.Match object; span=(37, 84), match='no sig..."
4550,CXR PA\n\nCompared with previous study: Jan 2...,"['<s> cxr pa <\\s>', '<s> compared with previo...",0,1,-1.0,"<re.Match object; span=(8, 59), match='no sign..."
4552,CXR PA upright\n\n Compared with previous che...,"['<s> cxr pa upright <\\s>', '<s> compared wit...",0,1,-1.0,"<re.Match object; span=(24, 67), match='no cha..."
4553,CXR PA upright\n\n Compared with previous che...,"['<s> cxr pa upright <\\s>', '<s> compared wit...",0,1,-1.0,"<re.Match object; span=(24, 64), match='no cha..."


In [905]:
# create report list
pleural_report_u = createReportList(pleural_df_u)
pleural_report_u[0]

['<s> chest pa upright <\\s>',
 '<s> follow up film compared to film on 22 3 2005 <\\s>',
 '<s> no significant changed of rt.pleural effusion opacity of rll <\\s>',
 '<s> l.t lung is clear <\\s>',
 '<s> remaining thickening of rt.paratracheal soft tissue is also shown <\\s>',
 '<s> heart is mild prominent in size <\\s>',
 '<s> tortuous thoracic aorta is seen <\\s>',
 '<s> ds <\\s>']

Find N-grams Pattern

In [906]:
pleural_bigram_u = createPatternNgrams(pleural_report_u, 2, pleural_keyword)
list(pleural_bigram_u.items())[:20]

[(('pleural', 'effusion'), 2275),
 (('change', 'of'), 1867),
 (('<s>', 'no'), 1680),
 (('effusion', '<\\s>'), 1348),
 (('no', 'change'), 1095),
 (('right', 'pleural'), 1079),
 (('no', 'significant'), 814),
 (('significant', 'change'), 748),
 (('left', 'pleural'), 746),
 (('of', 'right'), 615),
 (('costophrenic', 'angle'), 568),
 (('of', 'left'), 435),
 (('pleural', 'thickening'), 365),
 (('left', 'costophrenic'), 346),
 (('<s>', 'the'), 337),
 (('there', 'is'), 334),
 (('effusion', 'is'), 330),
 (('clear', '<\\s>'), 312),
 (('is', 'no'), 304),
 (('angle', 'is'), 287)]

In [907]:
pleural_trigram_u = createPatternNgrams(pleural_report_u, 3, pleural_keyword)
list(pleural_trigram_u.items())[:20]

[(('pleural', 'effusion', '<\\s>'), 1296),
 (('no', 'change', 'of'), 1062),
 (('right', 'pleural', 'effusion'), 1050),
 (('no', 'significant', 'change'), 748),
 (('significant', 'change', 'of'), 739),
 (('left', 'pleural', 'effusion'), 723),
 (('<s>', 'no', 'change'), 652),
 (('<s>', 'no', 'significant'), 603),
 (('of', 'right', 'pleural'), 502),
 (('change', 'of', 'right'), 427),
 (('of', 'left', 'pleural'), 324),
 (('pleural', 'effusion', 'is'), 319),
 (('left', 'costophrenic', 'angle'), 317),
 (('there', 'is', 'no'), 299),
 (('costophrenic', 'angle', 'is'), 278),
 (('change', 'of', 'left'), 276),
 (('<s>', 'there', 'is'), 262),
 (('right', 'costophrenic', 'angle'), 239),
 (('pleural', 'effusion', 'and'), 238),
 (('bilateral', 'pleural', 'effusion'), 222)]

## Unlabel

Prepare Data

In [908]:
# choose reports
pleural_df = selectData(pleural_data, 'Pleural Effusion', None)
pleural_df

,Reports,Report List,Pleural Effusion BERT Labeler,Pleural Effusion Inspectra Labeler,Pleural Effusion Ground Truth,Pleural Effusion Key Detect
3,CHEST\n\nRt hydropneumothorax is seen.\nLeft l...,"['<s> chest <\\s>', '<s> rt hydropneumothorax ...",0,1,NaN,NaN
6,CHEST\n\nThere is both upper lobes infiltratio...,"['<s> chest <\\s>', '<s> there is both upper l...",1,0,NaN,NaN
13,CHEST.;\nLoss of right breast shadow.\nMassive...,"['<s> chest <\\s>', '<s> loss of right breast ...",1,0,NaN,NaN
15,CHEST\n\nS/P sternotomy.\nCompared to previous...,"['<s> chest <\\s>', '<s> s p sternotomy <\\s>'...",0,1,NaN,NaN
19,Chest ( PA upright view)\n As compared with ...,"['<s> chest pa upright view <\\s>', '<s> as co...",0,1,NaN,NaN
...,...,...,...,...,...,...
4502,CXR PA upright\n\nHistory : --\n\nComparison s...,"['<s> cxr pa upright <\\s>', '<s> history <\\s...",0,1,NaN,NaN
4504,CXR PA\n\nCompared with previous study: 07/12/...,"['<s> cxr pa <\\s>', '<s> compared with previo...",1,0,NaN,NaN
4545,Chest PA upright\n\nComparison: 3/4/2018\n\nFi...,"['<s> chest pa upright <\\s>', '<s> comparison...",0,1,NaN,NaN
4554,Chest PA upright \n\ncompared to film on 25/1/...,"['<s> chest pa upright <\\s>', '<s> compared t...",0,1,NaN,NaN


In [909]:
# create report list
pleural_report = createReportList(pleural_df)
pleural_report[0]

['<s> chest <\\s>',
 '<s> rt hydropneumothorax is seen <\\s>',
 '<s> left lung is clear <\\s>',
 '<s> no cardiomegaly is noted <\\s>']

In [910]:
# define keyword
pleural_keyword = r'pleural|costophrenic|effus|costoph|pleu'

Find N-grams Pattern

In [911]:
pleural_bigram = createPatternNgrams(pleural_report, 2)
list(pleural_bigram.items())[:20]

[(('<s>', 'no'), 347),
 (('<s>', 'the'), 257),
 (('<s>', 'there'), 229),
 (('change', 'of'), 229),
 (('there', 'is'), 208),
 (('intact', '<\\s>'), 169),
 (('noted', '<\\s>'), 160),
 (('no', 'change'), 158),
 (('seen', '<\\s>'), 157),
 (('bony', 'thorax'), 154),
 (('is', 'seen'), 131),
 (('is', 'noted'), 131),
 (('left', 'lung'), 130),
 (('at', 'right'), 130),
 (('<s>', 'bony'), 128),
 (('lung', '<\\s>'), 126),
 (('of', 'the'), 125),
 (('is', 'intact'), 118),
 (('lung', 'is'), 114),
 (('thorax', 'is'), 109)]

In [912]:
pleural_trigram = createPatternNgrams(pleural_report, 3)
list(pleural_trigram.items())[:20]

[(('<s>', 'there', 'is'), 195),
 (('no', 'change', 'of'), 122),
 (('is', 'intact', '<\\s>'), 118),
 (('is', 'seen', '<\\s>'), 115),
 (('is', 'noted', '<\\s>'), 113),
 (('bony', 'thorax', 'is'), 107),
 (('thorax', 'is', 'intact'), 98),
 (('<s>', 'bony', 'thorax'), 89),
 (('no', 'significant', 'change'), 75),
 (('lung', 'is', 'clear'), 74),
 (('is', 'clear', '<\\s>'), 74),
 (('<s>', 'no', 'cardiomegaly'), 72),
 (('<s>', 'no', 'change'), 65),
 (('left', 'lung', 'is'), 64),
 (('not', 'enlarged', '<\\s>'), 59),
 (('is', 'not', 'enlarged'), 58),
 (('<s>', 'findings', '<\\s>'), 58),
 (('compared', 'with', 'previous'), 55),
 (('<s>', 'the', 'heart'), 55),
 (('significant', 'change', 'of'), 54)]

In [913]:
pleural_fourgram = createPatternNgrams(pleural_report, 4)
list(pleural_fourgram.items())[:20]

[(('bony', 'thorax', 'is', 'intact'), 98),
 (('thorax', 'is', 'intact', '<\\s>'), 98),
 (('lung', 'is', 'clear', '<\\s>'), 65),
 (('<s>', 'bony', 'thorax', 'is'), 64),
 (('is', 'not', 'enlarged', '<\\s>'), 57),
 (('<s>', 'no', 'change', 'of'), 56),
 (('no', 'significant', 'change', 'of'), 54),
 (('heart', 'is', 'not', 'enlarged'), 51),
 (('<s>', 'the', 'heart', 'is'), 50),
 (('left', 'lung', 'is', 'clear'), 49),
 (('<s>', 'there', 'is', 'no'), 49),
 (('<s>', 'no', 'significant', 'change'), 45),
 (('<s>', 'left', 'lung', 'is'), 43),
 (('<s>', 'the', 'chest', 'shows'), 41),
 (('<s>', 'the', 'bony', 'thorax'), 39),
 (('the', 'bony', 'thorax', 'is'), 39),
 (('<s>', 'no', 'cardiomegaly', '<\\s>'), 38),
 (('compared', 'with', 'previous', 'chest'), 36),
 (('with', 'previous', 'chest', 'on'), 36),
 (('the', 'heart', 'is', 'not'), 36)]

In [914]:
pleural_bigramk = createPatternNgrams(pleural_report, 2, pleural_keyword)
list(pleural_bigramk.items())[:20]

[(('change', 'of'), 103),
 (('<s>', 'no'), 78),
 (('no', 'change'), 78),
 (('effusion', '<\\s>'), 69),
 (('right', 'pleural'), 65),
 (('pleural', 'thickening'), 63),
 (('left', 'pleural'), 58),
 (('there', 'is'), 50),
 (('<s>', 'there'), 46),
 (('of', 'the'), 43),
 (('of', 'right'), 40),
 (('<s>', 'left'), 31),
 (('bilateral', 'pleural'), 30),
 (('left', 'effusion'), 30),
 (('no', 'significant'), 29),
 (('<s>', 'minimal'), 29),
 (('pleural', 'efusion'), 28),
 (('of', 'left'), 27),
 (('significant', 'change'), 26),
 (('lung', '<\\s>'), 25)]

In [915]:
pleural_trigramk = createPatternNgrams(pleural_report, 3, pleural_keyword)
list(pleural_trigramk.items())[:20]

[(('no', 'change', 'of'), 76),
 (('<s>', 'there', 'is'), 43),
 (('<s>', 'no', 'change'), 38),
 (('no', 'significant', 'change'), 26),
 (('change', 'of', 'the'), 22),
 (('there', 'is', 'no'), 22),
 (('significant', 'change', 'of'), 21),
 (('<s>', 'no', 'significant'), 19),
 (('of', 'right', 'pleural'), 19),
 (('pleural', 'thickening', '<\\s>'), 18),
 (('<s>', 'the', 'chest'), 18),
 (('the', 'chest', 'shows'), 18),
 (('is', 'no', 'change'), 17),
 (('left', 'effusion', '<\\s>'), 16),
 (('of', 'the', 'right'), 14),
 (('pleural', 'thickening', 'at'), 14),
 (('effusion', 'compared', 'to'), 13),
 (('pleural', 'effsuion', '<\\s>'), 13),
 (('pleural', 'efusion', '<\\s>'), 13),
 (('change', 'of', 'left'), 13)]

In [916]:
pleural_fourgramk = createPatternNgrams(pleural_report, 4, pleural_keyword)
list(pleural_fourgramk.items())[:20]

[(('<s>', 'no', 'change', 'of'), 38),
 (('no', 'change', 'of', 'the'), 21),
 (('no', 'significant', 'change', 'of'), 21),
 (('<s>', 'there', 'is', 'no'), 20),
 (('<s>', 'no', 'significant', 'change'), 18),
 (('<s>', 'the', 'chest', 'shows'), 18),
 (('there', 'is', 'no', 'change'), 17),
 (('is', 'no', 'change', 'of'), 17),
 (('the', 'chest', 'shows', 'no'), 12),
 (('no', 'change', 'of', 'left'), 11),
 (('chest', 'shows', 'no', 'change'), 10),
 (('shows', 'no', 'change', 'of'), 10),
 (('no', 'change', 'of', 'right'), 9),
 (('of', 'the', 'right', 'effusion'), 8),
 (('change', 'of', 'left', 'effusion'), 8),
 (('change', 'of', 'right', 'pleural'), 8),
 (('change', 'of', 'the', 'right'), 7),
 (('<s>', 'minimal', 'left', 'pleural'), 7),
 (('left', 'pleural', 'efffusion', '<\\s>'), 7),
 (('amount', 'of', 'right', 'pleural'), 7)]

In [917]:
pleural_fivegramk = createPatternNgrams(pleural_report, 5, pleural_keyword)
list(pleural_fivegramk.items())[:20]

[(('<s>', 'there', 'is', 'no', 'change'), 17),
 (('there', 'is', 'no', 'change', 'of'), 17),
 (('<s>', 'no', 'significant', 'change', 'of'), 17),
 (('<s>', 'the', 'chest', 'shows', 'no'), 12),
 (('the', 'chest', 'shows', 'no', 'change'), 10),
 (('is', 'no', 'change', 'of', 'the'), 10),
 (('<s>', 'no', 'change', 'of', 'the'), 9),
 (('chest', 'shows', 'no', 'change', 'of'), 9),
 (('<s>', 'no', 'change', 'of', 'left'), 8),
 (('no', 'change', 'of', 'the', 'right'), 7),
 (('no', 'change', 'of', 'left', 'effusion'), 7),
 (('change', 'of', 'the', 'rt', 'effusion'), 6),
 (('change', 'of', 'the', 'right', 'effusion'), 6),
 (('no', 'change', 'of', 'right', 'pleural'), 6),
 (('<s>', 'no', 'change', 'of', 'right'), 6),
 (('change', 'of', 'left', 'effusion', '<\\s>'), 6),
 (('<s>', 'as', 'compared', 'with', 'prior'), 5),
 (('no', 'change', 'of', 'the', 'left'), 5),
 (('change', 'of', 'the', 'left', 'effusion'), 5),
 (('change', 'of', 'moderate', 'amount', 'of'), 5)]

In [918]:
pleural_sixgramk = createPatternNgrams(pleural_report, 6, pleural_keyword)
list(pleural_sixgramk.items())[:20]

[(('<s>', 'there', 'is', 'no', 'change', 'of'), 17),
 (('<s>', 'the', 'chest', 'shows', 'no', 'change'), 10),
 (('there', 'is', 'no', 'change', 'of', 'the'), 10),
 (('the', 'chest', 'shows', 'no', 'change', 'of'), 9),
 (('no', 'change', 'of', 'the', 'right', 'effusion'), 6),
 (('no', 'change', 'of', 'the', 'left', 'effusion'), 5),
 (('<s>', 'no', 'change', 'of', 'left', 'effusion'), 5),
 (('no', 'change', 'of', 'the', 'rt', 'effusion'), 5),
 (('no', 'change', 'of', 'left', 'effusion', '<\\s>'), 5),
 (('<s>', 'no', 'change', 'of', 'right', 'pleural'), 5),
 (('<s>', 'as', 'compared', 'with', 'prior', 'study'), 4),
 (('as', 'compared', 'with', 'prior', 'study', 'on'), 4),
 (('no', 'significant', 'change', 'of', 'moderate', 'amount'), 4),
 (('change', 'of', 'the', 'right', 'effusion', '<\\s>'), 4),
 (('is', 'no', 'change', 'of', 'the', 'right'), 4),
 (('<s>', 'as', 'compared', 'to', 'prior', 'film'), 4),
 (('as', 'compared', 'to', 'prior', 'film', 'on'), 4),
 (('change', 'of', 'moderate', 

In [919]:
pleural_sevengramk = createPatternNgrams(pleural_report, 7, pleural_keyword)
list(pleural_sevengramk.items())[:20]

[(('<s>', 'there', 'is', 'no', 'change', 'of', 'the'), 10),
 (('<s>', 'the', 'chest', 'shows', 'no', 'change', 'of'), 9),
 (('<s>', 'as', 'compared', 'with', 'prior', 'study', 'on'), 4),
 (('no', 'change', 'of', 'the', 'right', 'effusion', '<\\s>'), 4),
 (('there', 'is', 'no', 'change', 'of', 'the', 'right'), 4),
 (('is', 'no', 'change', 'of', 'the', 'right', 'effusion'), 4),
 (('<s>', 'as', 'compared', 'to', 'prior', 'film', 'on'), 4),
 (('no', 'change', 'of', 'the', 'rt', 'effusion', 'compared'), 4),
 (('change', 'of', 'the', 'rt', 'effusion', 'compared', 'to'), 4),
 (('the', 'chest', 'shows', 'no', 'change', 'of', 'moderate'), 3),
 (('no', 'change', 'of', 'the', 'left', 'effusion', 'compared'), 3),
 (('change', 'of', 'the', 'left', 'effusion', 'compared', 'to'), 3),
 (('<s>', 'no', 'change', 'of', 'the', 'left', 'effusion'), 3),
 (('<s>', 'no', 'significant', 'change', 'of', 'moderate', 'amount'), 3),
 (('no', 'significant', 'change', 'of', 'moderate', 'amount', 'of'), 3),
 (('incre

## Difference between Result 1 and 2

In [920]:
findDiffData('Pleural Effusion', pleural_data_old, pleural_data)

nan 0.0 ['<s> cxr pa upright <\\s>', '<s> by compared with previous chest on 20 7 2006 <\\s>', '<s> the chest shows no change of moderate effusion at right hemithorax <\\s>', '<s> the lungs are clear <\\s>', '<s> the left costophrenic angle is still clear <\\s>', '<s> the bony thorax is intact <\\s>']
nan -1.0 ['<s> chest <\\s>', '<s> no change of both lung infiltrationm and plerual effusion <\\s>', '<s> osteolytic lesion at ribs and scapular <\\s>', '<s> right apical cap is seen <\\s>', '<s> normal cardiothoracic ratio <\\s>']
0.0 -1.0 ['<s> chest pa <\\s>', '<s> as compared to previous study on dec 12 2009 <\\s>', '<s> no change of right plerual effusion <\\s>', '<s> no change of diffuse reticulonodular infiltration along right lung <\\s>', '<s> no left pleural effusion <\\s>', '<s> mild cardiomegaly <\\s>']
nan -1.0 ['<s> chest \\ <\\s>', '<s> no change of rul lesion and right plerual effusion <\\s>', '<s> prominent left periaortic region is also not change <\\s>', '<s> svc stent in

# Atelectasis

In [921]:
# define keyword
# atelec_keyword = r'atelec|collap|volume'
atelec_keyword = r'atelec'

### Ground truth 0

Prepare Data

In [922]:
# choose reports
atelec_df_0 = selectData(atelec_data, 'Atelectasis', 0.0)
atelec_df_0

,Reports,Report List,Atelectasis BERT Labeler,Atelectasis Inspectra Labeler,Atelectasis Ground Truth,Atelectasis Key Detect
48,"Chest:\nCompared with 6-5-2008, no chage of RM...","['<s> chest <\\s>', '<s> compared with 6 5 200...",1,0,0.0,"<re.Match object; span=(31, 85), match='no cha..."
412,CXR PA upright\n\nHistory : ---\n\nComparison ...,"['<s> cxr pa upright <\\s>', '<s> history <\\s...",1,0,0.0,"<re.Match object; span=(8, 83), match='no demo..."
473,CXR PA upright\n\nMinimal right lower lung op...,"['<s> cxr pa upright <\\s>', '<s> minimal righ...",1,0,0.0,"<re.Match object; span=(8, 89), match='no demo..."
498,"CHEST FILM PA, UPRIGHT VIEW\n\nCOMPARISON: 07/...","['<s> chest film pa upright view <\\s>', '<s> ...",0,1,0.0,"<re.Match object; span=(8, 64), match='no inte..."
509,CXR\n Known case CA lung S/P CMT.\n Signific...,"['<s> cxr <\\s>', '<s> known case ca lung s p ...",0,1,0.0,"<re.Match object; span=(69, 140), match='no me..."
549,"CHEST, PA UPRIGHT\nAs comparison to prior CXR ...","['<s> chest pa upright <\\s>', '<s> as compari...",1,0,0.0,"<re.Match object; span=(85, 167), match='no sh..."
617,"CXR PA upright\n\nHistory : ESRD, HT, chronic...","['<s> cxr pa upright <\\s>', '<s> history esrd...",1,0,0.0,"<re.Match object; span=(8, 83), match='no demo..."
621,CXR PA upright\n\n Compared with previous che...,"['<s> cxr pa upright <\\s>', '<s> compared wit...",1,0,0.0,"<re.Match object; span=(24, 87), match='no mil..."
624,CXR PA upright\n\n\nAs compared with prior stu...,"['<s> cxr pa upright <\\s>', '<s> as compared ...",1,0,0.0,"<re.Match object; span=(8, 53), match='no demo..."
756,CXR PA upright\n\nHistory : CA breast. \n\nCo...,"['<s> cxr pa upright <\\s>', '<s> history ca b...",1,0,0.0,"<re.Match object; span=(8, 83), match='no demo..."


In [923]:
# create report list
atelec_report_0 = createReportList(atelec_df_0)
atelec_report_0[0]

['<s> chest <\\s>',
 '<s> compared with 6 5 2008 no chage of rml infiltration and atelectasis <\\s>',
 '<s> normal cardiothoracic ratio <\\s>']

Find N-grams Pattern

In [924]:
atelec_bigram_0 = createPatternNgrams(atelec_report_0, 2, atelec_keyword)
list(atelec_bigram_0.items())[:20]

[(('<s>', 'no'), 24),
 (('atelectasis', '<\\s>'), 23),
 (('or', 'atelectasis'), 17),
 (('no', 'demonstrable'), 16),
 (('mass', 'or'), 16),
 (('pulmonary', 'infiltration'), 12),
 (('active', 'pulmonary'), 11),
 (('infiltration', 'mass'), 11),
 (('demonstrable', 'active'), 9),
 (('change', 'of'), 7),
 (('no', 'interval'), 5),
 (('interval', 'change'), 5),
 (('is', 'noted'), 4),
 (('demonstrable', 'mass'), 4),
 (('and', 'atelectasis'), 3),
 (('noted', '<\\s>'), 3),
 (('with', 'no'), 3),
 (('<s>', 'as'), 3),
 (('of', 'mediastinum'), 3),
 (('atelectasis', 'and'), 3)]

In [925]:
atelec_trigram_0 = createPatternNgrams(atelec_report_0, 3, atelec_keyword)
list(atelec_trigram_0.items())[:20]

[(('<s>', 'no', 'demonstrable'), 16),
 (('mass', 'or', 'atelectasis'), 16),
 (('or', 'atelectasis', '<\\s>'), 16),
 (('active', 'pulmonary', 'infiltration'), 11),
 (('pulmonary', 'infiltration', 'mass'), 11),
 (('infiltration', 'mass', 'or'), 11),
 (('no', 'demonstrable', 'active'), 9),
 (('demonstrable', 'active', 'pulmonary'), 9),
 (('no', 'interval', 'change'), 5),
 (('interval', 'change', 'of'), 5),
 (('no', 'demonstrable', 'mass'), 4),
 (('demonstrable', 'mass', 'or'), 4),
 (('<s>', 'no', 'interval'), 3),
 (('is', 'noted', '<\\s>'), 3),
 (('and', 'atelectasis', '<\\s>'), 2),
 (('no', 'demonstrable', 'other'), 2),
 (('demonstrable', 'other', 'active'), 2),
 (('other', 'active', 'pulmonary'), 2),
 (('change', 'of', 'rul'), 2),
 (('of', 'rul', 'atelectasis'), 2)]

### Ground truth 1

Prepare Data

In [926]:
# choose reports
atelec_df_1 = selectData(atelec_data, 'Atelectasis', 1.0)
atelec_df_1

,Reports,Report List,Atelectasis BERT Labeler,Atelectasis Inspectra Labeler,Atelectasis Ground Truth,Atelectasis Key Detect
4,CXR\n\nRUL ateletasis.\nFibrocalcific infiltra...,"['<s> cxr <\\s>', '<s> rul ateletasis <\\s>', ...",1,0,1.0,"<re.Match object; span=(8, 32), match='rul ate..."
7,"Chest PA upright;\n known case CA lung, post C...","['<s> chest pa upright <\\s>', '<s> known case...",1,0,1.0,"<re.Match object; span=(66, 90), match='lll at..."
23,"CHEST.;\n\nKnoWn case of Ewing's sarcoma,post ...","['<s> chest <\\s>', ""<s> known case of ewing's...",1,0,1.0,"<re.Match object; span=(4, 29), match='<s> ate..."
25,CHEST\n\nAs compared with previous film on 20/...,"['<s> chest <\\s>', '<s> as compared with prev...",0,1,1.0,"<re.Match object; span=(4, 76), match='<s> ate..."
65,Chest PA.\n Ateletasis of RUL.\n right pleur...,"['<s> chest pa <\\s>', '<s> ateletasis of rul ...",1,0,1.0,"<re.Match object; span=(4, 35), match='<s> ate..."
176,CXR (PA)\n\nThe LLL opacity with sign of LLL v...,"['<s> cxr pa <\\s>', '<s> the lll opacity with...",1,0,1.0,"<re.Match object; span=(126, 151), match='lll ..."
185,Chest PA upruight \n\nHistory: RUL atelectasis...,"['<s> chest pa upruight <\\s>', '<s> history r...",1,0,1.0,"<re.Match object; span=(16, 41), match='rul at..."
186,CXR PA upright\n\nNo demonstrable active pulmo...,"['<s> cxr pa upright <\\s>', '<s> no demonstra...",0,1,1.0,"<re.Match object; span=(4, 73), match='<s> ate..."
317,CXR(PA upright)\n\n Hx of endobronchial tumor...,"['<s> cxr pa upright <\\s>', '<s> hx of endobr...",1,0,1.0,"<re.Match object; span=(46, 89), match='lul at..."
433,CXR\n\nS/P sternotomy.\nResolving LLL atelecta...,"['<s> cxr <\\s>', '<s> s p sternotomy <\\s>', ...",1,0,1.0,"<re.Match object; span=(18, 43), match='lll at..."


In [927]:
# create report list
atelec_report_1 = createReportList(atelec_df_1)
atelec_report_1[0]

['<s> cxr <\\s>',
 '<s> rul ateletasis <\\s>',
 '<s> fibrocalcific infiltration at both upper lobe and rll <\\s>',
 '<s> cardiomegaly <\\s>',
 '<s> no pleural effusion <\\s>',
 '<s> bont thorax is i8ntact <\\s>']


Find N-grams Pattern

In [928]:
atelec_bigram_1 = createPatternNgrams(atelec_report_1, 2, atelec_keyword)
list(atelec_bigram_1.items())[:20]

[(('<s>', 'atelectasis'), 10),
 (('atelectasis', '<\\s>'), 6),
 (('atelectasis', 'at'), 5),
 (('atelectesis', '<\\s>'), 4),
 (('atelectasis', 'and'), 4),
 (('rul', 'atelectasis'), 4),
 (('<s>', 'resolving'), 4),
 (('resolving', 'of'), 4),
 (('of', 'rul'), 4),
 (('at', 'rul'), 4),
 (('rul', '<\\s>'), 4),
 (('is', 'seen'), 4),
 (('rll', 'atelectasis'), 4),
 (('the', 'left'), 3),
 (('lll', 'atelectesis'), 3),
 (('<s>', 'partial'), 3),
 (('partial', 'resolved'), 3),
 (('atelectasis', 'is'), 3),
 (('seen', '<\\s>'), 3),
 (('at', 'lll'), 2)]

In [929]:
atelec_trigram_1 = createPatternNgrams(atelec_report_1, 3, atelec_keyword)
list(atelec_trigram_1.items())[:20]

[(('<s>', 'atelectasis', 'at'), 5),
 (('resolving', 'of', 'rul'), 4),
 (('at', 'rul', '<\\s>'), 4),
 (('lll', 'atelectesis', '<\\s>'), 3),
 (('atelectasis', 'at', 'rul'), 3),
 (('<s>', 'partial', 'resolved'), 3),
 (('rll', 'atelectasis', 'is'), 3),
 (('is', 'seen', '<\\s>'), 3),
 (('<s>', 'atelectasis', 'and'), 2),
 (('<s>', 'resolving', 'lll'), 2),
 (('resolving', 'lll', 'atelectasis'), 2),
 (('lll', 'atelectasis', '<\\s>'), 2),
 (('<s>', 'resolving', 'of'), 2),
 (('of', 'rul', 'atelectasis'), 2),
 (('rul', 'atelectasis', 'as'), 2),
 (('atelectasis', 'as', 'well'), 2),
 (('as', 'well', 'as'), 2),
 (('well', 'as', 'resolving'), 2),
 (('as', 'resolving', 'of'), 2),
 (('of', 'rul', 'interstitial'), 2)]

## Ground truth -1

Prepare Data

In [930]:
# choose reports
atelec_df_u = selectData(atelec_data, 'Atelectasis', -1.0)
atelec_df_u

,Reports,Report List,Atelectasis BERT Labeler,Atelectasis Inspectra Labeler,Atelectasis Ground Truth,Atelectasis Key Detect
3,"CXR\n Compare with film on 6/5/05, less expan...","['<s> cxr <\\s>', '<s> compare with film on 6 ...",0,1,-1.0,"<re.Match object; span=(8, 55), match='no chan..."
5,CXR PA upright\nHistory of ovarian cyst. \n\nS...,"['<s> cxr pa upright <\\s>', '<s> history of o...",0,1,-1.0,"<re.Match object; span=(14, 65), match='plate ..."
6,CXR PA upright\n By compared with previous ch...,"['<s> cxr pa upright <\\s>', '<s> by compared ...",0,1,-1.0,"<re.Match object; span=(22, 81), match='some a..."
9,CHEST\n\nPlate atelecsis at Rt lower lobe.\nFi...,"['<s> chest <\\s>', '<s> plate atelecsis at rt...",1,0,-1.0,"<re.Match object; span=(8, 50), match='plate a..."
11,CHEST\n\nCompared to previous film on 28/07/20...,"['<s> chest <\\s>', '<s> compared to previous ...",0,1,-1.0,"<re.Match object; span=(8, 113), match='no sig..."
...,...,...,...,...,...,...
2158,CXR PA upright\n\nHistory : RA with 1st hypot...,"['<s> cxr pa upright <\\s>', '<s> history ra w...",0,1,-1.0,"<re.Match object; span=(8, 71), match='no chan..."
2159,CXR(PA)\n\n Compare to CXR on 15/5/2015 and 1...,"['<s> cxr pa <\\s>', '<s> compare to cxr on 15...",0,1,-1.0,"<re.Match object; span=(8, 81), match='no chan..."
2162,Chest PA upright\n \n No pulmonary infiltrat...,"['<s> chest pa upright <\\s>', '<s> no pulmona...",1,0,-1.0,"<re.Match object; span=(8, 82), match='plate a..."
2163,CHEST FILM\n\nNo pulmonary infiltration is see...,"['<s> chest film <\\s>', '<s> no pulmonary inf...",1,0,-1.0,"<re.Match object; span=(8, 74), match='plate l..."


In [931]:
# create report list
atelec_report_u = createReportList(atelec_df_u)
atelec_report_u[0]

['<s> cxr <\\s>',
 '<s> compare with film on 6 5 05 less expansion of bilateral lungs are noted and possibly causing more prominent of interstitial thickening at lll <\\s>',
 '<s> no change of plate atelectasis at rll <\\s>',
 '<s> no cardiomegaly <\\s>',
 '<s> mildly prominent aorta is noted <\\s>',
 '<s> no pleural effusion <\\s>',
 '<s> intact bony thorax <\\s>']

Find N-grams Pattern

In [932]:
atelec_bigram_u = createPatternNgrams(atelec_report_u, 2, atelec_keyword)
list(atelec_bigram_u.items())[:20]

[(('atelectasis', 'at'), 658),
 (('change', 'of'), 515),
 (('<s>', 'no'), 480),
 (('plate', 'atelectasis'), 465),
 (('no', 'change'), 395),
 (('at', 'left'), 223),
 (('<s>', 'plate'), 222),
 (('of', 'plate'), 212),
 (('at', 'lll'), 198),
 (('no', 'significant'), 152),
 (('at', 'right'), 148),
 (('lung', '<\\s>'), 140),
 (('significant', 'change'), 124),
 (('lll', '<\\s>'), 111),
 (('plate', 'like'), 110),
 (('changed', 'of'), 109),
 (('like', 'atelectasis'), 108),
 (('lower', 'lung'), 107),
 (('of', 'atelectasis'), 95),
 (('noted', '<\\s>'), 94)]

In [933]:
atelec_trigram_u = createPatternNgrams(atelec_report_u, 3, atelec_keyword)
list(atelec_trigram_u.items())[:20]

[(('plate', 'atelectasis', 'at'), 392),
 (('no', 'change', 'of'), 385),
 (('<s>', 'no', 'change'), 276),
 (('atelectasis', 'at', 'left'), 200),
 (('of', 'plate', 'atelectasis'), 196),
 (('atelectasis', 'at', 'lll'), 176),
 (('change', 'of', 'plate'), 156),
 (('<s>', 'plate', 'atelectasis'), 125),
 (('no', 'significant', 'change'), 124),
 (('significant', 'change', 'of'), 119),
 (('plate', 'like', 'atelectasis'), 108),
 (('<s>', 'no', 'significant'), 105),
 (('atelectasis', 'at', 'right'), 102),
 (('like', 'atelectasis', 'at'), 99),
 (('at', 'lll', '<\\s>'), 97),
 (('no', 'changed', 'of'), 87),
 (('is', 'noted', '<\\s>'), 83),
 (('change', 'of', 'atelectasis'), 80),
 (('<s>', 'plate', 'like'), 76),
 (('at', 'left', 'lower'), 74)]

## Unlabel

Prepare Data

In [934]:
# choose reports
atelec_df = selectData(atelec_data, 'Atelectasis', None)
atelec_df

,Reports,Report List,Atelectasis BERT Labeler,Atelectasis Inspectra Labeler,Atelectasis Ground Truth,Atelectasis Key Detect
0,CXR\n Suboptimal inspiration is noted.\n Inc...,"['<s> cxr <\\s>', '<s> suboptimal inspiration ...",1,0,NaN,NaN
1,Chest film\nReticulonodular infiltration both ...,"['<s> chest film <\\s>', '<s> reticulonodular ...",1,0,NaN,NaN
2,CXR PA upright\nFibropathy and atelectasis at ...,"['<s> cxr pa upright <\\s>', '<s> fibropathy a...",0,1,NaN,NaN
8,CXR \n\nHistory: none\nComparison: 31.5.2005\...,"['<s> cxr <\\s>', '<s> history none <\\s>', '<...",1,0,NaN,NaN
10,CXR\n\nCardiomegaly with increase pulmonary va...,"['<s> cxr <\\s>', '<s> cardiomegaly with incre...",1,0,NaN,NaN
...,...,...,...,...,...,...
2164,Chest PA upright\n Compare to prior study date...,"['<s> chest pa upright <\\s>', '<s> compare to...",1,0,NaN,NaN
2165,CXR PA UPRIGHT\nFINDINGS:\nIncrease in amount ...,"['<s> cxr pa upright <\\s>', '<s> findings <\\...",1,0,NaN,NaN
2166,CXR PA UPRIGHT\n\nFINDINGS:\nFurther increase ...,"['<s> cxr pa upright <\\s>', '<s> findings <\\...",1,0,NaN,NaN
2168,CXR PA\n\nCompared study: 18/04/2018\nSuspicio...,"['<s> cxr pa <\\s>', '<s> compared study 18 04...",1,0,NaN,NaN


In [935]:
# create report list
atelec_report = createReportList(atelec_df)
atelec_report[0]

['<s> cxr <\\s>',
 '<s> suboptimal inspiration is noted <\\s>',
 '<s> increased lung marking at both basal lungs are noted and possibly due to collapse lung markings from not full inspiration <\\s>',
 '<s> no definite lung nifltration or nodule is seen <\\s>',
 '<s> heart size cannot be evaluate <\\s>']


Find N-grams Pattern

In [936]:
atelec_bigram = createPatternNgrams(atelec_report, 2)
list(atelec_bigram.items())[:20]

[(('<s>', 'no'), 1146),
 (('<s>', 'the'), 729),
 (('noted', '<\\s>'), 692),
 (('is', 'seen'), 610),
 (('is', 'noted'), 605),
 (('seen', '<\\s>'), 603),
 (('pleural', 'effusion'), 464),
 (('change', 'of'), 400),
 (('clear', '<\\s>'), 384),
 (('pulmonary', 'infiltration'), 339),
 (('both', 'costophrenic'), 332),
 (('effusion', '<\\s>'), 283),
 (('at', 'right'), 281),
 (('cardiomegaly', 'is'), 280),
 (('are', 'clear'), 275),
 (('<s>', 'findings'), 271),
 (('degenerative', 'change'), 270),
 (('costophrenic', 'angles'), 269),
 (('angles', 'are'), 267),
 (('left', 'lung'), 261)]

In [937]:
atelec_trigram = createPatternNgrams(atelec_report, 3)
list(atelec_trigram.items())[:20]

[(('is', 'noted', '<\\s>'), 533),
 (('is', 'seen', '<\\s>'), 529),
 (('are', 'clear', '<\\s>'), 273),
 (('costophrenic', 'angles', 'are'), 260),
 (('pleural', 'effusion', '<\\s>'), 254),
 (('both', 'costophrenic', 'angles'), 248),
 (('<s>', 'no', 'cardiomegaly'), 237),
 (('<s>', 'both', 'costophrenic'), 198),
 (('is', 'intact', '<\\s>'), 169),
 (('cardiomegaly', 'is', 'seen'), 168),
 (('pulmonary', 'infiltration', 'or'), 166),
 (('bony', 'thorax', 'is'), 163),
 (('no', 'cardiomegaly', 'is'), 159),
 (('calcified', 'aortic', 'knob'), 157),
 (('<s>', 'there', 'is'), 155),
 (('degenerative', 'change', 'of'), 153),
 (('thorax', 'is', 'intact'), 150),
 (('<s>', 'degenerative', 'change'), 146),
 (('angles', 'are', 'clear'), 144),
 (('no', 'pleural', 'effusion'), 139)]

In [938]:
atelec_fourgram = createPatternNgrams(atelec_report, 4)
list(atelec_fourgram.items())[:20]

[(('both', 'costophrenic', 'angles', 'are'), 242),
 (('cardiomegaly', 'is', 'seen', '<\\s>'), 167),
 (('<s>', 'no', 'cardiomegaly', 'is'), 159),
 (('bony', 'thorax', 'is', 'intact'), 150),
 (('thorax', 'is', 'intact', '<\\s>'), 150),
 (('angles', 'are', 'clear', '<\\s>'), 143),
 (('costophrenic', 'angles', 'are', 'clear'), 138),
 (('<s>', 'degenerative', 'change', 'of'), 137),
 (('<s>', 'no', 'pleural', 'effusion'), 136),
 (('<s>', 'no', 'blunting', 'of'), 134),
 (('<s>', 'calcified', 'aortic', 'knob'), 132),
 (('no', 'blunting', 'of', 'the'), 128),
 (('is', 'not', 'enlarged', '<\\s>'), 124),
 (('visualized', 'bony', 'structures', 'are'), 123),
 (('<s>', 'the', 'visualized', 'bony'), 121),
 (('<s>', 'both', 'costophrenic', 'angles'), 120),
 (('the', 'visualized', 'bony', 'structures'), 120),
 (('calcified', 'aortic', 'knob', 'is'), 114),
 (('heart', 'is', 'not', 'enlarged'), 113),
 (('pulmonary', 'infiltration', 'or', 'nodule'), 110)]

In [939]:
atelec_bigramk = createPatternNgrams(atelec_report, 2, atelec_keyword)
list(atelec_bigramk.items())[:20]

[(('atelectasis', 'at'), 11),
 (('at', 'rul'), 11),
 (('and', 'atelectasis'), 9),
 (('of', 'right'), 8),
 (('atelectasis', '<\\s>'), 6),
 (('rul', '<\\s>'), 5),
 (('atelectesis', '<\\s>'), 5),
 (('rml', 'atelectasis'), 5),
 (('at', 'lul'), 4),
 (('at', 'right'), 4),
 (('pleural', 'effusion'), 4),
 (('right', 'lung'), 4),
 (('lung', '<\\s>'), 4),
 (('<s>', 'no'), 4),
 (('no', 'change'), 4),
 (('change', 'of'), 4),
 (('atelectasis', 'is'), 4),
 (('<s>', 'resolved'), 3),
 (('atelectasis', 'of'), 3),
 (('mass', 'and'), 3)]

In [940]:
atelec_trigramk = createPatternNgrams(atelec_report, 3, atelec_keyword)
list(atelec_trigramk.items())[:20]

[(('and', 'atelectasis', 'at'), 8),
 (('atelectasis', 'at', 'rul'), 8),
 (('at', 'rul', '<\\s>'), 5),
 (('<s>', 'no', 'change'), 4),
 (('no', 'change', 'of'), 4),
 (('at', 'rul', 'and'), 3),
 (('at', 'lul', '<\\s>'), 3),
 (('<s>', 'pulmonary', 'infiltration'), 3),
 (('<s>', 'there', 'is'), 3),
 (('of', 'right', 'lung'), 3),
 (('rml', 'atelectasis', 'is'), 3),
 (('<s>', 'resolved', 'atelectasis'), 2),
 (('mass', 'and', 'atelectasis'), 2),
 (('right', 'pleural', 'effusion'), 2),
 (('to', 'previous', 'film'), 2),
 (('<s>', 'fibroinfiltration', 'and'), 2),
 (('fibroinfiltration', 'and', 'atelectasis'), 2),
 (('atelectasis', 'of', 'right'), 2),
 (('right', 'lung', '<\\s>'), 2),
 (('compressive', 'atelectasis', '<\\s>'), 2)]

In [941]:
atelec_fourgramk = createPatternNgrams(atelec_report, 4, atelec_keyword)
list(atelec_fourgramk.items())[:20]

[(('and', 'atelectasis', 'at', 'rul'), 6),
 (('atelectasis', 'at', 'rul', '<\\s>'), 5),
 (('<s>', 'no', 'change', 'of'), 4),
 (('mass', 'and', 'atelectasis', 'at'), 2),
 (('atelectasis', 'at', 'rul', 'and'), 2),
 (('<s>', 'fibroinfiltration', 'and', 'atelectasis'), 2),
 (('atelectasis', 'of', 'right', 'lung'), 2),
 (('of', 'right', 'lung', '<\\s>'), 2),
 (('platatelectasis', 'at', 'right', 'lower'), 2),
 (('at', 'right', 'lower', 'lung'), 2),
 (('right', 'lower', 'lung', 'with'), 2),
 (('lower', 'lung', 'with', 'minimal'), 2),
 (('lung', 'with', 'minimal', 'blunting'), 2),
 (('with', 'minimal', 'blunting', 'of'), 2),
 (('minimal', 'blunting', 'of', 'right'), 2),
 (('blunting', 'of', 'right', 'costophrenic'), 2),
 (('of', 'right', 'costophrenic', 'angle'), 2),
 (('right', 'costophrenic', 'angle', 'from'), 2),
 (('costophrenic', 'angle', 'from', 'minimal'), 2),
 (('angle', 'from', 'minimal', 'pleural'), 2)]

In [942]:
atelec_fivegramk = createPatternNgrams(atelec_report, 5, atelec_keyword)
list(atelec_fivegramk.items())[:20]

[(('and', 'atelectasis', 'at', 'rul', '<\\s>'), 4),
 (('mass', 'and', 'atelectasis', 'at', 'rul'), 2),
 (('and', 'atelectasis', 'at', 'rul', 'and'), 2),
 (('atelectasis', 'of', 'right', 'lung', '<\\s>'), 2),
 (('platatelectasis', 'at', 'right', 'lower', 'lung'), 2),
 (('at', 'right', 'lower', 'lung', 'with'), 2),
 (('right', 'lower', 'lung', 'with', 'minimal'), 2),
 (('lower', 'lung', 'with', 'minimal', 'blunting'), 2),
 (('lung', 'with', 'minimal', 'blunting', 'of'), 2),
 (('with', 'minimal', 'blunting', 'of', 'right'), 2),
 (('minimal', 'blunting', 'of', 'right', 'costophrenic'), 2),
 (('blunting', 'of', 'right', 'costophrenic', 'angle'), 2),
 (('of', 'right', 'costophrenic', 'angle', 'from'), 2),
 (('right', 'costophrenic', 'angle', 'from', 'minimal'), 2),
 (('costophrenic', 'angle', 'from', 'minimal', 'pleural'), 2),
 (('angle', 'from', 'minimal', 'pleural', 'thickening'), 2),
 (('from', 'minimal', 'pleural', 'thickening', '<\\s>'), 2),
 (('<s>', 'no', 'change', 'of', 'fibroatelect

In [943]:
atelec_sixgramk = createPatternNgrams(atelec_report, 6, atelec_keyword)
list(atelec_sixgramk.items())[:20]

[(('mass', 'and', 'atelectasis', 'at', 'rul', '<\\s>'), 2),
 (('platatelectasis', 'at', 'right', 'lower', 'lung', 'with'), 2),
 (('at', 'right', 'lower', 'lung', 'with', 'minimal'), 2),
 (('right', 'lower', 'lung', 'with', 'minimal', 'blunting'), 2),
 (('lower', 'lung', 'with', 'minimal', 'blunting', 'of'), 2),
 (('lung', 'with', 'minimal', 'blunting', 'of', 'right'), 2),
 (('with', 'minimal', 'blunting', 'of', 'right', 'costophrenic'), 2),
 (('minimal', 'blunting', 'of', 'right', 'costophrenic', 'angle'), 2),
 (('blunting', 'of', 'right', 'costophrenic', 'angle', 'from'), 2),
 (('of', 'right', 'costophrenic', 'angle', 'from', 'minimal'), 2),
 (('right', 'costophrenic', 'angle', 'from', 'minimal', 'pleural'), 2),
 (('costophrenic', 'angle', 'from', 'minimal', 'pleural', 'thickening'), 2),
 (('angle', 'from', 'minimal', 'pleural', 'thickening', '<\\s>'), 2),
 (('<s>', 'fibropathy', 'and', 'atelectasis', 'at', 'rul'), 1),
 (('fibropathy', 'and', 'atelectasis', 'at', 'rul', '<\\s>'), 1),


In [944]:
# define shorter keyword
atelec_keyword = r'aele'

In [945]:
atelec_bigramk = createPatternNgrams(atelec_report, 2, atelec_keyword)
list(atelec_bigramk.items())[:20]

[(('<s>', 'no'), 2),
 (('no', 'change'), 2),
 (('change', 'of'), 2),
 (('of', 'aelectasis'), 2),
 (('aelectasis', 'with'), 2),
 (('with', 'bronchiectasis'), 2),
 (('bronchiectasis', 'at'), 2),
 (('at', 'lll'), 2),
 (('lll', '<\\s>'), 2)]

## Difference between Result 1 and 2

In [946]:
findDiffData('Atelectasis', atelec_data_old, atelec_data)

nan -1.0 ['<s> chest <\\s>', '<s> history of ca lung <\\s>', '<s> compare with cxr on 9 5 2006 <\\s>', '<s> no significant change of right pleural effusion with large rt. lung mass causing mediastinal shifting to the left side <\\s>', '<s> possible subsegmental atelectasis at rul <\\s>', '<s> left lung is clear no left pleural effusion <\\s>']
nan -1.0 ['<s> cxr pa upright <\\s>', '<s> unchanged partially atelecatis at right upper lung and few nodule shadow at lul <\\s>', '<s> as compared with study on 5 4 2004 <\\s>', '<s> new faint opacity at lingular segment of left lung and right basal lung are noted possibly pneumonia <\\s>', '<s> normal cardiothoracic ratio <\\s>', '<s> bony thorax is intact <\\s>', '<s> both costophrenic sulci hili and diaphragm are not remarkable <\\s>']
nan -1.0 ['<s> cxr pa upright <\\s>', '<s> unchanged partially atelecatis at right upper lung and few nodule shadow at lul <\\s>', '<s> as compared with study on 16 6 2009 <\\s>', '<s> the rest of lung is clear

# Lung Lesion

In [947]:
# define keyword
lesion_keyword = r'nodule|mass'

## Ground truth 0

Prepare Data

In [948]:
# choose reports
lesion_df_0 = selectData(lesion_data, 'Lung Lesion', 0.0)
lesion_df_0

,Reports,Report List,Lung Lesion BERT Labeler,Lung Lesion Inspectra Labeler,Lung Lesion Ground Truth,Lung Lesion Key Detect
8,CXR\n\nHistory of goiter\n\nMild hyperinflatio...,"['<s> cxr <\\s>', '<s> history of goiter <\\s>...",1,0,0.0,"<re.Match object; span=(8, 27), match='no pulm..."
70,Chest PA upright\n Known case CA breast S/P CM...,"['<s> chest pa upright <\\s>', '<s> known case...",1,0,0.0,"<re.Match object; span=(8, 27), match='no pulm..."
72,CXR (PA upright)\nHx: Lt.parotidf tumor. Preo...,"['<s> cxr pa upright <\\s>', '<s> hx lt.paroti...",1,0,0.0,"<re.Match object; span=(8, 53), match='no defi..."
100,Chest film\nReticulonodular infiltration both ...,"['<s> chest film <\\s>', '<s> reticulonodular ...",1,0,0.0,"<re.Match object; span=(8, 24), match='no defi..."
119,Chest PA upright;\n compared to filmon 13/5/20...,"['<s> chest pa upright <\\s>', '<s> compared t...",0,1,0.0,"<re.Match object; span=(8, 37), match='no new ..."
...,...,...,...,...,...,...
34201,CXR PA upright\n\n There is no definite pulmo...,"['<s> cxr pa upright <\\s>', '<s> there is no ...",0,1,0.0,"<re.Match object; span=(17, 61), match='no def..."
34227,CXR PA upright\n\nHistory : ESRD \n\nCompari...,"['<s> cxr pa upright <\\s>', '<s> history esrd...",0,1,0.0,"<re.Match object; span=(8, 58), match='no demo..."
34267,CXR PA UPRIGHT \n\nFINDINGS:\n No pulmonary i...,"['<s> cxr pa upright <\\s>', '<s> findings <\\...",0,1,0.0,"<re.Match object; span=(8, 43), match='no pulm..."
34294,CXR\n\nCompared to prior film on 9 december 17...,"['<s> cxr <\\s>', '<s> compared to prior film ...",0,1,0.0,"<re.Match object; span=(61, 89), match='no def..."


In [949]:
# create report list
lesion_report_0 = createReportList(lesion_df_0)
lesion_report_0[0]

['<s> cxr <\\s>',
 '<s> history of goiter <\\s>',
 '<s> mild hyperinflation is noted bilaterally <\\s>',
 '<s> blunting of right costophrenic angle could be small pleural effusion or pleural thickening <\\s>',
 '<s> calcification at right lower lobe close to lateral chest wall <\\s>',
 '<s> interstitial thickening both lungs <\\s>',
 '<s> no pulmonary nodule is demonstrated <\\s>']

Find N-grams Pattern

In [950]:
lesion_bigram_0 = createPatternNgrams(lesion_report_0, 2, lesion_keyword)
list(lesion_bigram_0.items())[:20]

[(('<s>', 'no'), 900),
 (('pulmonary', 'infiltration'), 757),
 (('infiltration', 'or'), 477),
 (('or', 'nodule'), 410),
 (('active', 'pulmonary'), 385),
 (('no', 'demonstrable'), 328),
 (('mass', 'or'), 310),
 (('is', 'noted'), 303),
 (('noted', '<\\s>'), 301),
 (('or', 'atelectasis'), 299),
 (('atelectasis', '<\\s>'), 295),
 (('nodule', 'is'), 293),
 (('infiltration', 'mass'), 277),
 (('demonstrable', 'active'), 262),
 (('no', 'detectable'), 224),
 (('no', 'definite'), 216),
 (('nodule', '<\\s>'), 205),
 (('nodule', 'at'), 185),
 (('detectable', 'pulmonary'), 167),
 (('change', 'of'), 163)]

In [951]:
lesion_trigram_0 = createPatternNgrams(lesion_report_0, 3, lesion_keyword)
list(lesion_trigram_0.items())[:20]

[(('pulmonary', 'infiltration', 'or'), 415),
 (('infiltration', 'or', 'nodule'), 400),
 (('active', 'pulmonary', 'infiltration'), 378),
 (('<s>', 'no', 'demonstrable'), 325),
 (('mass', 'or', 'atelectasis'), 299),
 (('or', 'atelectasis', '<\\s>'), 294),
 (('is', 'noted', '<\\s>'), 287),
 (('pulmonary', 'infiltration', 'mass'), 277),
 (('infiltration', 'mass', 'or'), 264),
 (('demonstrable', 'active', 'pulmonary'), 262),
 (('no', 'demonstrable', 'active'), 261),
 (('or', 'nodule', 'is'), 242),
 (('nodule', 'is', 'noted'), 189),
 (('no', 'detectable', 'pulmonary'), 166),
 (('detectable', 'pulmonary', 'infiltration'), 154),
 (('<s>', 'findings', 'no'), 152),
 (('findings', 'no', 'detectable'), 150),
 (('no', 'definite', 'pulmonary'), 127),
 (('or', 'nodule', '<\\s>'), 119),
 (('is', 'seen', '<\\s>'), 108)]

## Ground truth 1

Prepare Data

In [952]:
# choose reports
lesion_df_1 = selectData(lesion_data, 'Lung Lesion', 1.0)
lesion_df_1

,Reports,Report List,Lung Lesion BERT Labeler,Lung Lesion Inspectra Labeler,Lung Lesion Ground Truth,Lung Lesion Key Detect
170,Chest PA upright;\n Advanced CA breast with lu...,"['<s> chest pa upright <\\s>', '<s> advanced c...",1,0,1.0,"<re.Match object; span=(31, 41), match='lll no..."
289,Chest PA urpight;\n known case CA lung S/P ch...,"['<s> chest pa urpight <\\s>', '<s> known case...",1,0,1.0,"<re.Match object; span=(62, 70), match='rul ma..."
290,Chest PA upright \nknown case small cell lung ...,"['<s> chest pa upright <\\s>', '<s> known case...",1,0,1.0,"<re.Match object; span=(27, 35), match='lul ma..."
306,CXR\n Hx of CA lung S/P CMT.\n Ill-defined...,"['<s> cxr <\\s>', '<s> hx of ca lung s p cmt <...",0,1,1.0,"<re.Match object; span=(12, 75), match='define..."
332,CXR PA UPRIGHT\n\n A 67 years old woman w...,"['<s> cxr pa upright <\\s>', '<s> a 67 years o...",1,0,1.0,"<re.Match object; span=(8, 18), match='lul nod..."
...,...,...,...,...,...,...
33359,CXR PA\n\nThere is increased amount of left ma...,"['<s> cxr pa <\\s>', '<s> there is increased a...",1,0,1.0,"<re.Match object; span=(37, 46), match='left m..."
33368,CXR : PA\n\nHistory: RUL nodule S/P surgery\nS...,"['<s> cxr pa <\\s>', '<s> history rul nodule s...",1,0,1.0,"<re.Match object; span=(16, 26), match='rul no..."
33584,CXR PA upright\n\nNo demonstrable active pulmo...,"['<s> cxr pa upright <\\s>', '<s> no demonstra...",0,1,1.0,"<re.Match object; span=(37, 45), match='rll ma..."
33585,CXR : PA\n\nHistory: RUL nodule S/P surgery\nN...,"['<s> cxr pa <\\s>', '<s> history rul nodule s...",1,0,1.0,"<re.Match object; span=(16, 26), match='rul no..."


In [953]:
# create report list
lesion_report_1 = createReportList(lesion_df_1)
lesion_report_1[0]

['<s> chest pa upright <\\s>',
 '<s> advanced ca breast with lung and bone metastasis <\\s>',
 '<s> no striking changed of reticulonoduular infiltration in lt.lung <\\s>',
 '<s> there is not well seen lll nodule <\\s>',
 '<s> rtlung is relative clear <\\s>',
 '<s> heart is mild enlared <\\s>',
 '<s> ds <\\s>']

Find N-grams Pattern

In [954]:
lesion_bigram_1 = createPatternNgrams(lesion_report_1, 2, lesion_keyword)
list(lesion_bigram_1.items())[:20]

[(('<s>', 'history'), 23),
 (('pleural', 'effusion'), 19),
 (('<s>', 'no'), 17),
 (('rul', 'nodule'), 17),
 (('nodule', 'is'), 15),
 (('<s>', 'massive'), 14),
 (('nodule', 'at'), 14),
 (('nodule', '<\\s>'), 13),
 (('at', 'left'), 13),
 (('mass', '<\\s>'), 12),
 (('right', 'pleural'), 12),
 (('effusion', '<\\s>'), 11),
 (('rul', 'mass'), 10),
 (('is', 'noted'), 10),
 (('lul', 'nodule'), 10),
 (('is', 'seen'), 10),
 (('<s>', 'a'), 10),
 (('<s>', 'mass'), 10),
 (('there', 'is'), 9),
 (('massive', 'right'), 9)]

In [955]:
lesion_trigram_1 = createPatternNgrams(lesion_report_1, 3, lesion_keyword)
list(lesion_trigram_1.items())[:20]

[(('right', 'pleural', 'effusion'), 12),
 (('pleural', 'effusion', '<\\s>'), 9),
 (('massive', 'right', 'pleural'), 9),
 (('<s>', 'there', 'is'), 8),
 (('<s>', 'massive', 'right'), 8),
 (('nodule', 'is', 'seen'), 6),
 (('left', 'pleural', 'effusion'), 6),
 (('<s>', 'history', 'rul'), 6),
 (('is', 'seen', '<\\s>'), 6),
 (('this', 'study', '<\\s>'), 5),
 (('pleural', 'effusion', 'with'), 5),
 (('pleural', 'effusion', 'is'), 5),
 (('of', 'rul', 'nodule'), 5),
 (('<s>', 'nodule', 'at'), 5),
 (('history', 'rul', 'nodule'), 5),
 (('nipple', 'shadow', '<\\s>'), 5),
 (('<s>', 'history', 'of'), 4),
 (('<s>', 'no', 'definite'), 4),
 (('nodule', 'is', 'noted'), 4),
 (('rll', 'mass', '<\\s>'), 4)]

## Ground truth -1

Prepare Data

In [956]:
# choose reports
lesion_df_u = selectData(lesion_data, 'Lung Lesion', -1.0)
lesion_df_u

,Reports,Report List,Lung Lesion BERT Labeler,Lung Lesion Inspectra Labeler,Lung Lesion Ground Truth,Lung Lesion Key Detect
7,CXR\n\n No significant change of a RUL nodu...,"['<s> cxr <\\s>', '<s> no significant change o...",0,1,-1.0,"<re.Match object; span=(8, 45), match='no sign..."
13,Chest;\n\n Small calcified nodule at right lo...,"['<s> chest <\\s>', '<s> small calcified nodul...",0,1,-1.0,"<re.Match object; span=(8, 30), match='small c..."
23,CXR\n\nHistory of CA rectum\n\nAs compared to ...,"['<s> cxr <\\s>', '<s> history of ca rectum <\...",0,1,-1.0,"<re.Match object; span=(63, 101), match='no ch..."
25,CXR PA upright\n The chest shows multiple cal...,"['<s> cxr pa upright <\\s>', '<s> the chest sh...",0,1,-1.0,"<re.Match object; span=(24, 48), match='multip..."
31,"CXR\n Compare with film on 21/4/05, no change...","['<s> cxr <\\s>', '<s> compare with film on 21...",1,0,-1.0,"<re.Match object; span=(37, 89), match='no cha..."
...,...,...,...,...,...,...
34319,CXR PA upright\n\nComparison to prior radiogra...,"['<s> cxr pa upright <\\s>', '<s> comparison t...",0,1,-1.0,"<re.Match object; span=(30, 45), match='multip..."
34328,Chest PA upright\n\n Overall no significant c...,"['<s> chest pa upright <\\s>', '<s> overall no...",0,1,-1.0,"<re.Match object; span=(16, 72), match='no sig..."
34331,CXR PA UPRIGHT\n\nThere is a small calcified n...,"['<s> cxr pa upright <\\s>', '<s> there is a s...",0,1,-1.0,"<re.Match object; span=(19, 41), match='small ..."
34336,CXR PA upright\n\n The chest shows no change ...,"['<s> cxr pa upright <\\s>', '<s> the chest sh...",0,1,-1.0,"<re.Match object; span=(24, 56), match='no cha..."


In [957]:
# create report list
lesion_report_u = createReportList(lesion_df_u)
lesion_report_u[0]

['<s> cxr <\\s>',
 '<s> no significant change of a rul nodule as compare with film on 5 2 05 <\\s>',
 '<s> rest lungs are clear <\\s>',
 '<s> no pleural effusion <\\s>',
 '<s> cardiomegaly with markedly dilated and calcified aorta suggested aneurysm <\\s>']

Find N-grams Pattern

In [958]:
lesion_bigram_u = createPatternNgrams(lesion_report_u, 2, lesion_keyword)
list(lesion_bigram_u.items())[:20]

[(('change', 'of'), 5784),
 (('<s>', 'no'), 3600),
 (('nodule', 'at'), 3584),
 (('no', 'change'), 3304),
 (('no', 'significant'), 2435),
 (('significant', 'change'), 2230),
 (('calcified', 'nodule'), 2149),
 (('at', 'right'), 2147),
 (('nodules', 'at'), 1992),
 (('<s>', 'the'), 1734),
 (('the', 'chest'), 1536),
 (('chest', 'shows'), 1531),
 (('shows', 'no'), 1459),
 (('at', 'left'), 1344),
 (('at', 'both'), 1214),
 (('lung', '<\\s>'), 1175),
 (('at', 'rul'), 1174),
 (('of', 'multiple'), 1092),
 (('calcific', 'nodule'), 1073),
 (('of', 'a'), 1063)]

In [959]:
lesion_trigram_u = createPatternNgrams(lesion_report_u, 3, lesion_keyword)
list(lesion_trigram_u.items())[:20]

[(('no', 'change', 'of'), 3249),
 (('no', 'significant', 'change'), 2227),
 (('significant', 'change', 'of'), 2204),
 (('calcified', 'nodule', 'at'), 1711),
 (('<s>', 'no', 'significant'), 1656),
 (('<s>', 'the', 'chest'), 1534),
 (('the', 'chest', 'shows'), 1531),
 (('chest', 'shows', 'no'), 1429),
 (('shows', 'no', 'change'), 1320),
 (('<s>', 'no', 'change'), 1193),
 (('change', 'of', 'multiple'), 994),
 (('nodule', 'at', 'right'), 928),
 (('change', 'of', 'a'), 888),
 (('nodules', 'at', 'both'), 874),
 (('calcific', 'nodule', 'at'), 776),
 (('at', 'both', 'lungs'), 705),
 (('both', 'lungs', '<\\s>'), 704),
 (('nodule', 'at', 'rul'), 661),
 (('small', 'calcified', 'nodule'), 654),
 (('<s>', 'there', 'is'), 648)]

## Unlabel

Prepare Data

In [960]:
# choose reports
lesion_df = selectData(lesion_data, 'Lung Lesion', None)
lesion_df

,Reports,Report List,Lung Lesion BERT Labeler,Lung Lesion Inspectra Labeler,Lung Lesion Ground Truth,Lung Lesion Key Detect
0,CXR (PA upright)\n\nReticulonodular infiltrati...,"['<s> cxr pa upright <\\s>', '<s> reticulonodu...",1,0,NaN,NaN
1,Chest:-\nPa chest study reveals fibronodular i...,"['<s> chest <\\s>', '<s> pa chest study reveal...",1,0,NaN,NaN
2,CHEST :\nP.A. upright view .\nFibronodular inf...,"['<s> chest <\\s>', '<s> p.a <\\s>', '<s> upri...",1,0,NaN,NaN
3,CXR\n No previous film to be compared.\n RLL...,"['<s> cxr <\\s>', '<s> no previous film to be ...",1,0,NaN,NaN
4,CXR PA upright\n The chest shows mild reticul...,"['<s> cxr pa upright <\\s>', '<s> the chest sh...",1,0,NaN,NaN
...,...,...,...,...,...,...
34343,CXR PA upright\n\n DIffuse reticulonodular in...,"['<s> cxr pa upright <\\s>', '<s> diffuse reti...",1,0,NaN,NaN
34344,CXR PA \n\nDiffuse reticulonodular infiltratio...,"['<s> cxr pa <\\s>', '<s> diffuse reticulonodu...",1,0,NaN,NaN
34345,Chest PA upright \n\nknown CA breast\n\nSurgic...,"['<s> chest pa upright <\\s>', '<s> known ca b...",1,0,NaN,NaN
34346,CXR(PA)\n \n Retilocalcified nodules of bot...,"['<s> cxr pa <\\s>', '<s> retilocalcified nodu...",0,1,NaN,NaN


In [961]:
# create report list
lesion_report = createReportList(lesion_df)
lesion_report[0]

['<s> cxr pa upright <\\s>',
 '<s> reticulonodular infiltration at both upper lobes <\\s>',
 '<s> the heart and great vessels appear normal <\\s>',
 '<s> no pleural effusion <\\s>',
 '<s> bony thorax is intact <\\s>',
 '<s> imp tb is considered <\\s>',
 '<s> clinical and previous film correlation are <\\s>',
 '<s> suggested <\\s>']

Find N-grams Pattern

In [962]:
lesion_bigram = createPatternNgrams(lesion_report, 2)
list(lesion_bigram.items())[:20]

[(('<s>', 'no'), 21816),
 (('bony', 'thorax'), 12005),
 (('infiltration', 'at'), 11837),
 (('pleural', 'effusion'), 11355),
 (('<s>', 'the'), 10701),
 (('intact', '<\\s>'), 9446),
 (('reticulonodular', 'infiltration'), 9283),
 (('thorax', 'is'), 9013),
 (('is', 'intact'), 8531),
 (('<s>', 'bony'), 8052),
 (('effusion', '<\\s>'), 7706),
 (('is', 'seen'), 7590),
 (('seen', '<\\s>'), 7443),
 (('noted', '<\\s>'), 7067),
 (('clear', '<\\s>'), 7006),
 (('no', 'pleural'), 6884),
 (('both', 'costophrenic'), 6536),
 (('<s>', 'both'), 6382),
 (('<s>', 'normal'), 6305),
 (('is', 'noted'), 6298)]

In [963]:
lesion_trigram = createPatternNgrams(lesion_report, 3)
list(lesion_trigram.items())[:20]

[(('bony', 'thorax', 'is'), 8995),
 (('is', 'intact', '<\\s>'), 8529),
 (('thorax', 'is', 'intact'), 8095),
 (('pleural', 'effusion', '<\\s>'), 7274),
 (('<s>', 'bony', 'thorax'), 7242),
 (('no', 'pleural', 'effusion'), 6849),
 (('reticulonodular', 'infiltration', 'at'), 6575),
 (('is', 'seen', '<\\s>'), 6497),
 (('<s>', 'no', 'pleural'), 6492),
 (('are', 'clear', '<\\s>'), 5775),
 (('<s>', 'both', 'costophrenic'), 5657),
 (('is', 'noted', '<\\s>'), 5533),
 (('<s>', 'no', 'cardiomegaly'), 5498),
 (('costophrenic', 'angles', 'are'), 4425),
 (('<s>', 'there', 'is'), 4273),
 (('both', 'costophrenic', 'angles'), 3974),
 (('angles', 'are', 'clear'), 3745),
 (('<s>', 'the', 'heart'), 3359),
 (('costophrenic', 'sulci', 'are'), 3303),
 (('no', 'cardiomegaly', '<\\s>'), 3268)]

In [964]:
lesion_fourgram = createPatternNgrams(lesion_report, 4)
list(lesion_fourgram.items())[:20]

[(('thorax', 'is', 'intact', '<\\s>'), 8093),
 (('bony', 'thorax', 'is', 'intact'), 8087),
 (('<s>', 'bony', 'thorax', 'is'), 6724),
 (('<s>', 'no', 'pleural', 'effusion'), 6460),
 (('no', 'pleural', 'effusion', '<\\s>'), 4987),
 (('both', 'costophrenic', 'angles', 'are'), 3817),
 (('angles', 'are', 'clear', '<\\s>'), 3738),
 (('costophrenic', 'angles', 'are', 'clear'), 3653),
 (('<s>', 'both', 'costophrenic', 'angles'), 3201),
 (('<s>', 'normal', 'cardiothoracic', 'ratio'), 2787),
 (('are', 'not', 'remarkable', '<\\s>'), 2714),
 (('<s>', 'no', 'cardiomegaly', '<\\s>'), 2670),
 (('normal', 'cardiothoracic', 'ratio', '<\\s>'), 2592),
 (('<s>', 'both', 'costophrenic', 'sulci'), 2435),
 (('<s>', 'reticulonodular', 'infiltration', 'at'), 2400),
 (('<s>', 'no', 'cardiomegaly', 'is'), 2390),
 (('is', 'not', 'enlarged', '<\\s>'), 2364),
 (('<s>', 'the', 'bony', 'thorax'), 2109),
 (('the', 'bony', 'thorax', 'is'), 2085),
 (('reticulonodular', 'infiltration', 'at', 'both'), 2064)]

In [965]:
lesion_bigramk = createPatternNgrams(lesion_report, 2, lesion_keyword)
list(lesion_bigramk.items())[:20]

[(('nodule', 'at'), 77),
 (('mass', '<\\s>'), 63),
 (('<s>', 'history'), 62),
 (('at', 'right'), 44),
 (('at', 'left'), 40),
 (('at', 'rul'), 39),
 (('nodule', '<\\s>'), 35),
 (('is', 'seen'), 35),
 (('pulmonary', 'nodule'), 35),
 (('<s>', 'the'), 31),
 (('mass', 'at'), 31),
 (('nodules', 'at'), 29),
 (('soft', 'tissue'), 29),
 (('<s>', 'there'), 28),
 (('seen', 'at'), 25),
 (('lower', 'lung'), 25),
 (('there', 'is'), 24),
 (('thyroid', 'nodule'), 23),
 (('is', 'noted'), 23),
 (('at', 'lul'), 22)]

In [966]:
lesion_trigramk = createPatternNgrams(lesion_report, 3, lesion_keyword)
list(lesion_trigramk.items())[:20]

[(('<s>', 'there', 'is'), 22),
 (('thyroid', 'nodule', '<\\s>'), 20),
 (('nodule', 'at', 'rul'), 20),
 (('<s>', 'clinical', 'history'), 18),
 (('nodule', 'at', 'left'), 16),
 (('is', 'noted', '<\\s>'), 16),
 (('pulmonary', 'nodule', 'at'), 16),
 (('density', 'nodule', 'at'), 16),
 (('at', 'left', 'lower'), 14),
 (('calcific', 'density', 'nodule'), 14),
 (('<s>', 'history', 'a'), 14),
 (('at', 'rul', '<\\s>'), 14),
 (('soft', 'tissue', 'mass'), 13),
 (('nodule', 'at', 'right'), 13),
 (('calcified', 'granuloma', '<\\s>'), 13),
 (('nodule', 'is', 'seen'), 13),
 (('is', 'seen', 'at'), 13),
 (('at', 'right', 'lower'), 12),
 (('left', 'lower', 'lung'), 12),
 (('pleural', 'effusion', '<\\s>'), 11)]

In [967]:
lesion_fourgramk = createPatternNgrams(lesion_report, 4, lesion_keyword)
list(lesion_fourgramk.items())[:20]

[(('calcific', 'density', 'nodule', 'at'), 14),
 (('nodule', 'is', 'seen', 'at'), 13),
 (('at', 'left', 'lower', 'lung'), 12),
 (('<s>', 'calcified', 'pulmonary', 'nodule'), 10),
 (('history', 'thyroid', 'nodule', '<\\s>'), 9),
 (('nodules', 'are', 'seen', 'at'), 9),
 (('<s>', 'a', 'calcific', 'density'), 9),
 (('a', 'calcific', 'density', 'nodule'), 9),
 (('nodule', 'at', 'left', 'lower'), 9),
 (('pulmonary', 'nodule', 'is', 'seen'), 9),
 (('compatible', 'with', 'history', 'of'), 8),
 (('soft', 'tissue', 'mass', 'at'), 8),
 (('at', 'right', 'lower', 'lung'), 8),
 (('<s>', 'obscuration', 'of', 'mass'), 8),
 (('density', 'nodule', 'at', 'left'), 7),
 (('nodule', 'at', 'rul', '<\\s>'), 7),
 (('obscuration', 'of', 'mass', 'lesion'), 7),
 (('calcified', 'pulmonary', 'nodule', 'is'), 7),
 (('<s>', 'history', 'thyroid', 'nodule'), 6),
 (('is', 'likely', 'calcified', 'granuloma'), 6)]

In [968]:
lesion_fivegramk = createPatternNgrams(lesion_report, 5, lesion_keyword)
list(lesion_fivegramk.items())[:20]

[(('<s>', 'a', 'calcific', 'density', 'nodule'), 9),
 (('a', 'calcific', 'density', 'nodule', 'at'), 9),
 (('nodule', 'at', 'left', 'lower', 'lung'), 9),
 (('pulmonary', 'nodule', 'is', 'seen', 'at'), 9),
 (('<s>', 'obscuration', 'of', 'mass', 'lesion'), 7),
 (('<s>', 'calcified', 'pulmonary', 'nodule', 'is'), 7),
 (('calcified', 'pulmonary', 'nodule', 'is', 'seen'), 7),
 (('is', 'likely', 'calcified', 'granuloma', '<\\s>'), 6),
 (('this', 'region', 'is', 'possible', '<\\s>'), 6),
 (('<s>', 'history', 'thyroid', 'nodule', '<\\s>'), 5),
 (('density', 'nodule', 'at', 'left', 'lower'), 5),
 (('<s>', 'history', 'breast', 'mass', '<\\s>'), 5),
 (('nodule', 'is', 'seen', 'at', 'rll'), 5),
 (('is', 'seen', 'at', 'rll', '<\\s>'), 5),
 (('calcific', 'density', 'nodule', 'at', 'left'), 5),
 (('<s>', 'clinical', 'history', 'thyroid', 'nodule'), 4),
 (('clinical', 'history', 'thyroid', 'nodule', '<\\s>'), 4),
 (('<s>', 'right', 'paratracheal', 'mass', 'is'), 4),
 (('right', 'paratracheal', 'mass',

In [969]:
lesion_sixgramk = createPatternNgrams(lesion_report, 6, lesion_keyword)
list(lesion_sixgramk.items())[:20]

[(('<s>', 'a', 'calcific', 'density', 'nodule', 'at'), 9),
 (('<s>', 'calcified', 'pulmonary', 'nodule', 'is', 'seen'), 7),
 (('calcified', 'pulmonary', 'nodule', 'is', 'seen', 'at'), 7),
 (('density', 'nodule', 'at', 'left', 'lower', 'lung'), 5),
 (('nodule', 'is', 'seen', 'at', 'rll', '<\\s>'), 5),
 (('<s>', 'clinical', 'history', 'thyroid', 'nodule', '<\\s>'), 4),
 (('<s>', 'right', 'paratracheal', 'mass', 'is', 'noted'), 4),
 (('right', 'paratracheal', 'mass', 'is', 'noted', 'not'), 4),
 (('paratracheal', 'mass', 'is', 'noted', 'not', 'changed'), 4),
 (('mass', 'is', 'noted', 'not', 'changed', '<\\s>'), 4),
 (('nodules', 'are', 'seen', 'at', 'lul', '<\\s>'), 4),
 (('<s>', 'obscuration', 'of', 'mass', 'lesion', 'at'), 4),
 (('pulmonary', 'nodule', 'is', 'seen', 'at', 'rll'), 4),
 (('nodule', 'at', 'left', 'lower', 'lung', 'field'), 4),
 (('as', 'compared', 'to', 'prior', 'film', 'on'), 4),
 (('a', 'calcific', 'density', 'nodule', 'at', 'right'), 3),
 (('<s>', 'a', 'round', 'shape', 

## Difference between Result 1 and 2

In [970]:
findDiffData('Lung Lesion', lesion_data_old, lesion_data)

nan -1.0 ['<s> cxr pa upright <\\s>', '<s> no demonstrable active pulmonary infiltration <\\s>', '<s> calcified nodules are seen at rul are seen which may be calcified granuloma <\\s>', '<s> cardiac measurement cannot be well evaluated due to inadequate deep inspiration <\\s>', '<s> artherosclerotic changes of aorta <\\s>', '<s> bony thorax is intact <\\s>', '<s> both costophrenic sulci are not remarkable <\\s>']
nan -1.0 ['<s> chest <\\s>', '<s> no definite pulmonary infiltration <\\s>', '<s> calcific nodule at left middle lung field <\\s>', '<s> no cardiomegaly or pleural effusion <\\s>', '<s> bony thorax is intact <\\s>']
nan -1.0 ['<s> cxr pa upright <\\s>', '<s> increased amount of left pleural effusion when compare to previous film on 29 5 2005 <\\s>', '<s> calcified nodule is seen at rigth lower lung which may be calcified granuloma <\\s>']
nan -1.0 ['<s> cxr pa upright <\\s>', '<s> interstitial thickening at rll <\\s>', '<s> calcified nodule is seen at lll field is seen which m

# Inspectra Lung Opacity v1

In [971]:
# define keyword
infil_keyword = r'(infi|densi|opaci|'
conso_keyword = 'consoli|'
opaci_keyword = 'opaci|translu|air|mark|pattern|lung|reticu|scar|thick|densi|patchy)'
v1_keyword = infil_keyword + conso_keyword + opaci_keyword

## Ground truth 0

Prepare Data

In [972]:
# choose reports
v1_df_0 = selectData(v1_data, 'Inspectra Lung Opacity v1', 0.0)
v1_df_0

,Reports,Report List,Inspectra Lung Opacity v1 BERT Labeler,Inspectra Lung Opacity v1 Inspectra Labeler,Inspectra Lung Opacity v1 Ground Truth,Inspectra Lung Opacity v1 Key Detect
13,Chest film\n\nNo definite pulmonary infiltrati...,"['<s> chest film <\\s>', '<s> no definite pulm...",0,1,0.0,"<re.Match object; span=(8, 34), match='no defi..."
43,CXR (PA upright)\n\nNo definite pulmonary inf...,"['<s> cxr pa upright <\\s>', '<s> no definite ...",0,1,0.0,"<re.Match object; span=(8, 34), match='no defi..."
61,CXR PA upright\n The chest shows no definite ...,"['<s> cxr pa upright <\\s>', '<s> the chest sh...",0,1,0.0,"<re.Match object; span=(24, 50), match='no def..."
97,CXR \n\nHistory of chronic cough\n\nNo definit...,"['<s> cxr <\\s>', '<s> history of chronic coug...",0,1,0.0,"<re.Match object; span=(8, 34), match='no defi..."
101,CXR\n No pulmonary infiltration or nodule.\n ...,"['<s> cxr <\\s>', '<s> no pulmonary infiltrati...",0,1,0.0,"<re.Match object; span=(8, 25), match='no pulm..."
...,...,...,...,...,...,...
76632,CHEST\n\nNo cardiomegaly is noted.\nNo changhe...,"['<s> chest <\\s>', '<s> no cardiomegaly is no...",0,1,0.0,"<re.Match object; span=(8, 44), match='no chan..."
76645,Chest PA upright\n\nFindings:\nNo definite pul...,"['<s> chest pa upright <\\s>', '<s> findings <...",0,1,0.0,"<re.Match object; span=(8, 34), match='no defi..."
76649,Chest\nNo new active infiltration\nUnchanged i...,"['<s> chest <\\s>', '<s> no new active infiltr...",0,1,0.0,"<re.Match object; span=(8, 26), match='no new ..."
76651,CXR PA UPRIGHT\n\nFINDINGS : No detectable pu...,"['<s> cxr pa upright <\\s>', '<s> findings no ...",0,1,0.0,"<re.Match object; span=(17, 76), match='no det..."


In [973]:
# create report list
v1_report_0 = createReportList(v1_df_0)
v1_report_0[0]

['<s> chest film <\\s>',
 '<s> no definite pulmonary infiltration <\\s>',
 '<s> radiodensity line in rul likely skin fold <\\s>',
 '<s> cardiomegaly with dilated aorta <\\s>',
 '<s> right paratracheal density could be vascular shadow <\\s>',
 '<s> both costrophrenic angles are clear <\\s>',
 '<s> old fracture right 8th rib <\\s>']

Find N-grams Pattern

In [974]:
v1_bigram_0 = createPatternNgrams(v1_report_0, 2, v1_keyword)
list(v1_bigram_0.items())[:20]

[(('pulmonary', 'infiltration'), 30681),
 (('infiltration', 'or'), 15368),
 (('or', 'nodule'), 15124),
 (('infiltration', '<\\s>'), 14567),
 (('no', 'definite'), 14505),
 (('active', 'pulmonary'), 13994),
 (('definite', 'pulmonary'), 13712),
 (('there', 'is'), 13424),
 (('is', 'no'), 13063),
 (('<s>', 'there'), 13054),
 (('<s>', 'imp'), 12808),
 (('nodule', '<\\s>'), 12804),
 (('imp', 'no'), 12791),
 (('no', 'active'), 11771),
 (('noted', '<\\s>'), 4259),
 (('is', 'noted'), 3866),
 (('<s>', 'no'), 3754),
 (('<s>', 'findings'), 3095),
 (('infiltration', 'at'), 2980),
 (('no', 'detectable'), 2772)]

In [975]:
v1_trigram_0 = createPatternNgrams(v1_report_0, 3, v1_keyword)
list(v1_trigram_0.items())[:20]

[(('pulmonary', 'infiltration', 'or'), 15321),
 (('infiltration', 'or', 'nodule'), 15116),
 (('pulmonary', 'infiltration', '<\\s>'), 14371),
 (('active', 'pulmonary', 'infiltration'), 13985),
 (('no', 'definite', 'pulmonary'), 13708),
 (('definite', 'pulmonary', 'infiltration'), 13530),
 (('there', 'is', 'no'), 13056),
 (('<s>', 'there', 'is'), 13019),
 (('is', 'no', 'definite'), 12795),
 (('<s>', 'imp', 'no'), 12791),
 (('or', 'nodule', '<\\s>'), 12744),
 (('no', 'active', 'pulmonary'), 11725),
 (('imp', 'no', 'active'), 11133),
 (('is', 'noted', '<\\s>'), 3650),
 (('<s>', 'findings', 'no'), 2700),
 (('findings', 'no', 'detectable'), 2676),
 (('or', 'nodule', 'is'), 2128),
 (('nodule', 'is', 'noted'), 2079),
 (('fibrotic', 'infiltration', 'at'), 1799),
 (('is', 'seen', '<\\s>'), 1750)]

## Ground truth 1

Prepare Data

In [976]:
# choose reports
v1_df_1 = selectData(v1_data, 'Inspectra Lung Opacity v1', 1.0)
v1_df_1

,Reports,Report List,Inspectra Lung Opacity v1 BERT Labeler,Inspectra Lung Opacity v1 Inspectra Labeler,Inspectra Lung Opacity v1 Ground Truth,Inspectra Lung Opacity v1 Key Detect
0,CXR (PA upright)\n\nPatchy infiltration at RL...,"['<s> cxr pa upright <\\s>', '<s> patchy infil...",0,1,1.0,"<re.Match object; span=(0, 19), match='<s> <s>..."
1,CXR (PA upright)\n\nReticulonodular infiltrati...,"['<s> cxr pa upright <\\s>', '<s> reticulonodu...",0,1,1.0,"<re.Match object; span=(0, 28), match='<s> <s>..."
2,Chest:-\nPa chest study reveals fibronodular i...,"['<s> chest <\\s>', '<s> pa chest study reveal...",0,1,1.0,"<re.Match object; span=(0, 48), match='<s> <s>..."
3,CHEST :\nP.A. upright view .\nFibronodular inf...,"['<s> chest <\\s>', '<s> p.a <\\s>', '<s> upri...",0,1,1.0,"<re.Match object; span=(0, 25), match='<s> <s>..."
4,CHEST :\nP.A. upright view .\nFibronodular inf...,"['<s> chest <\\s>', '<s> p.a <\\s>', '<s> upri...",0,1,1.0,"<re.Match object; span=(0, 25), match='<s> <s>..."
...,...,...,...,...,...,...
76653,Chest PA upright \n\nknown CA breast\n\nSurgic...,"['<s> chest pa upright <\\s>', '<s> known ca b...",0,1,1.0,"<re.Match object; span=(0, 70), match='<s> <s>..."
76655,CXR PA\n\nThere is interstitial infiltation at...,"['<s> cxr pa <\\s>', '<s> there is interstitia...",0,1,1.0,"<re.Match object; span=(0, 34), match='<s> <s>..."
76657,CXR(PA)\n \n Reticulocalcified infiltration...,"['<s> cxr pa <\\s>', '<s> reticulocalcified in...",0,1,1.0,"<re.Match object; span=(0, 30), match='<s> <s>..."
76658,CXR(PA)\n\n IMP: Retioculonodular infiltratio...,"['<s> cxr pa <\\s>', '<s> imp retioculonodular...",0,1,1.0,"<re.Match object; span=(0, 33), match='<s> <s>..."


In [977]:
# create report list
v1_report_1 = createReportList(v1_df_1)
v1_report_1[0]

['<s> cxr pa upright <\\s>',
 '<s> patchy infiltration at rll lul and lll <\\s>',
 '<s> mild cardiomegaly <\\s>',
 '<s> no perihilar or mediastinal mass <\\s>',
 '<s> no pleural effusion <\\s>',
 '<s> bony thorax is intact <\\s>']

Find N-grams Pattern

In [978]:
v1_bigram_1 = createPatternNgrams(v1_report_1, 2, v1_keyword)
list(v1_bigram_1.items())[:20]

[(('infiltration', 'at'), 11782),
 (('at', 'both'), 6766),
 (('reticulonodular', 'infiltration'), 6480),
 (('at', 'right'), 5190),
 (('at', 'rul'), 4867),
 (('<s>', 'reticulonodular'), 4395),
 (('both', 'lungs'), 4232),
 (('noted', '<\\s>'), 4220),
 (('is', 'seen'), 4077),
 (('is', 'noted'), 3666),
 (('unremarkable', '<\\s>'), 3504),
 (('seen', '<\\s>'), 3497),
 (('pleural', 'thickening'), 3437),
 (('<s>', 'no'), 3435),
 (('at', 'left'), 3397),
 (('opacity', 'at'), 3176),
 (('<s>', 'there'), 3065),
 (('pulmonary', 'infiltration'), 3062),
 (('there', 'is'), 3003),
 (('<s>', 'the'), 2973)]

In [979]:
v1_trigram_1 = createPatternNgrams(v1_report_1, 3, v1_keyword)
list(v1_trigram_1.items())[:20]

[(('reticulonodular', 'infiltration', 'at'), 4485),
 (('infiltration', 'at', 'both'), 3590),
 (('<s>', 'reticulonodular', 'infiltration'), 3235),
 (('infiltration', 'at', 'rul'), 2677),
 (('<s>', 'there', 'is'), 2593),
 (('is', 'noted', '<\\s>'), 2491),
 (('is', 'seen', '<\\s>'), 2354),
 (('not', 'remarkable', '<\\s>'), 2266),
 (('are', 'not', 'remarkable'), 2237),
 (('at', 'both', 'upper'), 2142),
 (('at', 'both', 'lungs'), 2135),
 (('at', 'rul', '<\\s>'), 1985),
 (('costophrenic', 'sulci', 'are'), 1725),
 (('sulci', 'are', 'not'), 1721),
 (('both', 'upper', 'lobes'), 1689),
 (('interstitial', 'infiltration', 'at'), 1681),
 (('both', 'lungs', '<\\s>'), 1613),
 (('infiltration', 'at', 'right'), 1592),
 (('nodular', 'opacity', 'at'), 1546),
 (('both', 'costophrenic', 'sulci'), 1516)]

## Ground truth -1

Prepare Data

In [980]:
# choose reports
v1_df_u = selectData(v1_data, 'Inspectra Lung Opacity v1', -1.0)
v1_df_u

,Reports,Report List,Inspectra Lung Opacity v1 BERT Labeler,Inspectra Lung Opacity v1 Inspectra Labeler,Inspectra Lung Opacity v1 Ground Truth,Inspectra Lung Opacity v1 Key Detect
6,CXR PA upright\n The chest shows mild reticul...,"['<s> cxr pa upright <\\s>', '<s> the chest sh...",0,1,-1.0,"<re.Match object; span=(24, 36), match='mild r..."
11,CXR\n Multiple nodules are noted at left lung...,"['<s> cxr <\\s>', '<s> multiple nodules are no...",0,1,-1.0,"<re.Match object; span=(56, 70), match='sugges..."
12,Chest;\n\n\n No significant change of reticul...,"['<s> chest <\\s>', '<s> no significant change...",0,1,-1.0,"<re.Match object; span=(8, 84), match='no sign..."
14,Chest film\n\nUnchanged fibrocalcific infiltra...,"['<s> chest film <\\s>', '<s> unchanged fibroc...",0,1,-1.0,"<re.Match object; span=(8, 86), match='unchang..."
31,CXR\n\nHistory of rule out TB\n\nAs compared t...,"['<s> cxr <\\s>', '<s> history of rule out tb ...",0,1,-1.0,"<re.Match object; span=(65, 198), match='no si..."
...,...,...,...,...,...,...
76641,Chest CXR\n\nNo change of a tiny nodular opaci...,"['<s> chest cxr <\\s>', '<s> no change of a ti...",0,1,-1.0,"<re.Match object; span=(8, 64), match='no chan..."
76644,CXR PA UPRIGHT\nCompared with 27/09/2017\n\n- ...,"['<s> cxr pa upright <\\s>', '<s> compared wit...",0,1,-1.0,"<re.Match object; span=(8, 53), match='no sign..."
76648,CXR PA\n\nNo change of nodular shadow at left ...,"['<s> cxr pa <\\s>', '<s> no change of nodular...",0,1,-1.0,"<re.Match object; span=(8, 84), match='no chan..."
76654,"Chest:\nNo change of emphysematous lung, RUL a...","['<s> chest <\\s>', '<s> no change of emphysem...",0,1,-1.0,"<re.Match object; span=(8, 56), match='no chan..."


In [981]:
# create report list
v1_report_u = createReportList(v1_df_u)
v1_report_u[0]

['<s> cxr pa upright <\\s>',
 '<s> the chest shows mild reticulonodular infiltration at right apical lung <\\s>',
 '<s> the left lung is clear <\\s>',
 '<s> the heart is not enlarged <\\s>',
 '<s> both costophrenic angles are clear <\\s>',
 '<s> the bony thorax is intact <\\s>']

Find N-grams Pattern

In [982]:
v1_bigram_u = createPatternNgrams(v1_report_u, 2, v1_keyword)
list(v1_bigram_u.items())[:20]

[(('change', 'of'), 20067),
 (('<s>', 'no'), 15417),
 (('infiltration', 'at'), 12525),
 (('no', 'change'), 10765),
 (('no', 'significant'), 9998),
 (('significant', 'change'), 9365),
 (('at', 'right'), 7900),
 (('<s>', 'the'), 6746),
 (('at', 'both'), 6690),
 (('lung', '<\\s>'), 5005),
 (('at', 'left'), 4962),
 (('reticulonodular', 'infiltration'), 4695),
 (('unremarkable', '<\\s>'), 4331),
 (('at', 'rul'), 4320),
 (('pleural', 'thickening'), 4073),
 (('the', 'chest'), 3925),
 (('chest', 'shows'), 3914),
 (('noted', '<\\s>'), 3633),
 (('interstitial', 'infiltration'), 3618),
 (('opacity', 'at'), 3583)]

In [983]:
v1_trigram_u = createPatternNgrams(v1_report_u, 3, v1_keyword)
list(v1_trigram_u.items())[:20]

[(('no', 'change', 'of'), 10380),
 (('no', 'significant', 'change'), 9357),
 (('significant', 'change', 'of'), 8907),
 (('<s>', 'no', 'significant'), 7288),
 (('<s>', 'no', 'change'), 5123),
 (('<s>', 'the', 'chest'), 3924),
 (('the', 'chest', 'shows'), 3914),
 (('reticulonodular', 'infiltration', 'at'), 3475),
 (('infiltration', 'at', 'both'), 3278),
 (('chest', 'shows', 'no'), 3251),
 (('shows', 'no', 'change'), 3041),
 (('<s>', 'there', 'is'), 2933),
 (('infiltration', 'at', 'right'), 2628),
 (('is', 'noted', '<\\s>'), 2454),
 (('interstitial', 'infiltration', 'at'), 2347),
 (('infiltration', 'at', 'rul'), 2315),
 (('change', 'of', 'interstitial'), 2247),
 (('there', 'is', 'no'), 2054),
 (('of', 'reticulonodular', 'infiltration'), 1996),
 (('is', 'unremarkable', '<\\s>'), 1931)]

## Unlabel

Prepare Data

In [984]:
# choose reports
v1_df = selectData(v1_data, 'Inspectra Lung Opacity v1', None)
v1_df

,Reports,Report List,Inspectra Lung Opacity v1 BERT Labeler,Inspectra Lung Opacity v1 Inspectra Labeler,Inspectra Lung Opacity v1 Ground Truth,Inspectra Lung Opacity v1 Key Detect
438,CXR\n\nMinimal RUL interstitial inflitration i...,"['<s> cxr <\\s>', '<s> minimal rul interstitia...",0,1,NaN,NaN
534,CXR PA upright\n\nFibronfiltration at LUL\nFib...,"['<s> cxr pa upright <\\s>', '<s> fibronfiltra...",0,1,NaN,NaN
674,CHEST PA\n\nSoft tissue mass at the LUL region...,"['<s> chest pa <\\s>', '<s> soft tissue mass a...",1,0,NaN,NaN
1146,CHEST PA\n\n There is round lucency superimpos...,"['<s> chest pa <\\s>', '<s> there is round luc...",1,0,NaN,NaN
2415,CXR\n\nMinimal fibrotic iniltration in LUL.\nT...,"['<s> cxr <\\s>', '<s> minimal fibrotic iniltr...",0,1,NaN,NaN
...,...,...,...,...,...,...
75421,CXR (PA)\n\nIntersttiial oapcity are seen at b...,"['<s> cxr pa <\\s>', '<s> intersttiial oapcity...",0,1,NaN,NaN
75514,CXR (PA)\n\nAtelectasis and bronchiectasis of ...,"['<s> cxr pa <\\s>', '<s> atelectasis and bron...",0,1,NaN,NaN
76013,CXR\n\nPulmonary inflitration at LLL with smal...,"['<s> cxr <\\s>', '<s> pulmonary inflitration ...",0,1,NaN,NaN
76068,"Chest:PA,upright\nCardiomegaly is noted.\nPulm...","['<s> chest pa upright <\\s>', '<s> cardiomega...",1,0,NaN,NaN


In [985]:
# create report list
v1_report = createReportList(v1_df)
v1_report[0]

['<s> cxr <\\s>',
 '<s> minimal rul interstitial inflitration is detected <\\s>',
 '<s> no pleural effusion is seen <\\s>',
 '<s> no cardiomegaly <\\s>',
 '<s> the mediastinum and bony thorax are normal <\\s>']

In [986]:
v1_report[1:5]

[['<s> cxr pa upright <\\s>',
  '<s> fibronfiltration at lul <\\s>',
  '<s> fibrosis rll <\\s>',
  '<s> normal cardiothoracic ratio <\\s>',
  '<s> bony thorax is intact <\\s>',
  '<s> elevation of right hemidiaphragm <\\s>'],
 ['<s> chest pa <\\s>',
  '<s> soft tissue mass at the lul region with destructionof the left 3rd rib <\\s>',
  '<s> old fx of the right posteriro 6 7 th rib <\\s>',
  '<s> cavitating lesion at the periphery of the rul <\\s>',
  '<s> left effusion <\\s>'],
 ['<s> chest pa <\\s>',
  '<s> there is round lucency superimpose with the cardiac shadow could be hiatal hernia or morgagni hernia <\\s>'],
 ['<s> cxr <\\s>',
  '<s> minimal fibrotic iniltration in lul <\\s>',
  '<s> the heart and great vessels appear normal <\\s>',
  '<s> no pleural effusion <\\s>',
  '<s> bony thorax is intact <\\s>']]

Find N-grams Pattern

In [987]:
v1_bigram = createPatternNgrams(v1_report, 2)
list(v1_bigram.items())[:20]

[(('<s>', 'no'), 159),
 (('pleural', 'effusion'), 124),
 (('no', 'pleural'), 85),
 (('effusion', '<\\s>'), 84),
 (('<s>', 'normal'), 76),
 (('bony', 'thorax'), 75),
 (('<s>', 'the'), 63),
 (('is', 'noted'), 63),
 (('seen', '<\\s>'), 62),
 (('intact', '<\\s>'), 62),
 (('is', 'seen'), 58),
 (('<s>', 'mild'), 57),
 (('noted', '<\\s>'), 55),
 (('is', 'intact'), 48),
 (('cardiomegaly', '<\\s>'), 46),
 (('size', '<\\s>'), 44),
 (('<s>', 'cardiomegaly'), 43),
 (('thorax', 'is'), 41),
 (('change', 'of'), 41),
 (('<s>', 'bony'), 39)]

In [988]:
v1_trigram = createPatternNgrams(v1_report, 3)
list(v1_trigram.items())[:20]

[(('no', 'pleural', 'effusion'), 84),
 (('<s>', 'no', 'pleural'), 83),
 (('pleural', 'effusion', '<\\s>'), 81),
 (('is', 'intact', '<\\s>'), 48),
 (('is', 'seen', '<\\s>'), 46),
 (('is', 'noted', '<\\s>'), 44),
 (('bony', 'thorax', 'is'), 41),
 (('thorax', 'is', 'intact'), 41),
 (('<s>', 'mild', 'cardiomegaly'), 35),
 (('<s>', 'normal', 'heart'), 33),
 (('normal', 'heart', 'size'), 33),
 (('pleural', 'effusion', 'is'), 31),
 (('<s>', 'no', 'cardiomegaly'), 30),
 (('bony', 'thorax', '<\\s>'), 28),
 (('heart', 'size', '<\\s>'), 28),
 (('<s>', 'bony', 'thorax'), 26),
 (('both', 'costophrenic', 'angles'), 26),
 (('are', 'clear', '<\\s>'), 26),
 (('<s>', 'there', 'is'), 23),
 (('cardiomegaly', 'is', 'noted'), 23)]

In [989]:
v1_fourgram = createPatternNgrams(v1_report, 4)
list(v1_fourgram.items())[:20]

[(('<s>', 'no', 'pleural', 'effusion'), 82),
 (('no', 'pleural', 'effusion', '<\\s>'), 61),
 (('bony', 'thorax', 'is', 'intact'), 41),
 (('thorax', 'is', 'intact', '<\\s>'), 41),
 (('<s>', 'normal', 'heart', 'size'), 33),
 (('normal', 'heart', 'size', '<\\s>'), 27),
 (('<s>', 'bony', 'thorax', 'is'), 25),
 (('angles', 'are', 'clear', '<\\s>'), 23),
 (('costophrenic', 'angles', 'are', 'clear'), 22),
 (('no', 'pleural', 'effusion', 'is'), 20),
 (('effusion', 'is', 'seen', '<\\s>'), 20),
 (('<s>', 'no', 'cardiomegaly', '<\\s>'), 20),
 (('<s>', 'both', 'costophrenic', 'angles'), 20),
 (('both', 'costophrenic', 'angles', 'are'), 20),
 (('pleural', 'effusion', 'is', 'seen'), 19),
 (('<s>', 'mild', 'cardiomegaly', 'is'), 16),
 (('<s>', 'normal', 'bony', 'thorax'), 14),
 (('normal', 'bony', 'thorax', '<\\s>'), 14),
 (('<s>', 'the', 'heart', 'is'), 14),
 (('<s>', 'intact', 'bony', 'thorax'), 13)]

In [990]:
v1_bigramk = createPatternNgrams(v1_report, 2, v1_keyword)
list(v1_bigramk.items())[:20]

[(('<s>', 'peribronchial'), 2),
 (('peribronchial', 'thickeing'), 2),
 (('thickeing', 'at'), 1),
 (('at', 'rul'), 1),
 (('rul', '<\\s>'), 1),
 (('<s>', 'thicked'), 1),
 (('thicked', 'wall'), 1),
 (('wall', 'cavity'), 1),
 (('cavity', 'lesion'), 1),
 (('lesion', 'at'), 1),
 (('at', 'rll'), 1),
 (('rll', 'is'), 1),
 (('is', 'seen'), 1),
 (('seen', 'no'), 1),
 (('no', 'significant'), 1),
 (('significant', 'change'), 1),
 (('change', 'from'), 1),
 (('from', 'previous'), 1),
 (('previous', 'film'), 1),
 (('film', '<\\s>'), 1)]

In [991]:
v1_trigramk = createPatternNgrams(v1_report, 3, v1_keyword)
list(v1_trigramk.items())[:20]

[(('<s>', 'peribronchial', 'thickeing'), 1),
 (('peribronchial', 'thickeing', 'at'), 1),
 (('thickeing', 'at', 'rul'), 1),
 (('at', 'rul', '<\\s>'), 1),
 (('<s>', 'thicked', 'wall'), 1),
 (('thicked', 'wall', 'cavity'), 1),
 (('wall', 'cavity', 'lesion'), 1),
 (('cavity', 'lesion', 'at'), 1),
 (('lesion', 'at', 'rll'), 1),
 (('at', 'rll', 'is'), 1),
 (('rll', 'is', 'seen'), 1),
 (('is', 'seen', 'no'), 1),
 (('seen', 'no', 'significant'), 1),
 (('no', 'significant', 'change'), 1),
 (('significant', 'change', 'from'), 1),
 (('change', 'from', 'previous'), 1),
 (('from', 'previous', 'film'), 1),
 (('previous', 'film', '<\\s>'), 1),
 (('<s>', 'diffuse', 'peribronchial'), 1),
 (('diffuse', 'peribronchial', 'thickeing'), 1)]

In [992]:
v1_fourgramk = createPatternNgrams(v1_report, 4, v1_keyword)
list(v1_fourgramk.items())[:20]

[(('<s>', 'peribronchial', 'thickeing', 'at'), 1),
 (('peribronchial', 'thickeing', 'at', 'rul'), 1),
 (('thickeing', 'at', 'rul', '<\\s>'), 1),
 (('<s>', 'thicked', 'wall', 'cavity'), 1),
 (('thicked', 'wall', 'cavity', 'lesion'), 1),
 (('wall', 'cavity', 'lesion', 'at'), 1),
 (('cavity', 'lesion', 'at', 'rll'), 1),
 (('lesion', 'at', 'rll', 'is'), 1),
 (('at', 'rll', 'is', 'seen'), 1),
 (('rll', 'is', 'seen', 'no'), 1),
 (('is', 'seen', 'no', 'significant'), 1),
 (('seen', 'no', 'significant', 'change'), 1),
 (('no', 'significant', 'change', 'from'), 1),
 (('significant', 'change', 'from', 'previous'), 1),
 (('change', 'from', 'previous', 'film'), 1),
 (('from', 'previous', 'film', '<\\s>'), 1),
 (('<s>', 'diffuse', 'peribronchial', 'thickeing'), 1),
 (('diffuse', 'peribronchial', 'thickeing', '<\\s>'), 1),
 (('<s>', 'peribronchial', 'thickeingat', 'right'), 1),
 (('peribronchial', 'thickeingat', 'right', 'perihilar'), 1)]

In [993]:
# define shorter keyword
infil_keyword_2 = r'infi|densi|opaci|'
conso_keyword_2 = 'consoli|'
opaci_keyword_2 = 'opaci|translu|air|mark|pattern|lung|reticu|scar|thicken|densi|patchy'
v1_keyword_2 = infil_keyword_2 + conso_keyword_2 + opaci_keyword_2

In [994]:
v1_bigramk = createPatternNgrams(v1_report, 2, v1_keyword_2)
list(v1_bigramk.items())[:20]

[]

In [995]:
v1_trigramk = createPatternNgrams(v1_report, 3, v1_keyword_2)
list(v1_trigramk.items())[:20]

[]

In [996]:
v1_fourgramk = createPatternNgrams(v1_report, 4, v1_keyword_2)
list(v1_fourgramk.items())[:20]

[]

In [997]:
# define shorter keyword
infil_keyword_3 = r'inf|dens|opac|'
conso_keyword_3 = 'consol|'
opaci_keyword_3 = 'opaci|translu|air|mark|pattern|lung|reticu|scar|thick|densi|patchy'
v1_keyword_3 = infil_keyword_3 + conso_keyword_3 + opaci_keyword_3

In [998]:
v1_bigramk = createPatternNgrams(v1_report, 2, v1_keyword_3)
list(v1_bigramk.items())[:20]

[(('<s>', 'there'), 9),
 (('at', 'rul'), 7),
 (('interstitial', 'inflitration'), 6),
 (('at', 'the'), 6),
 (('inflitration', 'at'), 6),
 (('at', 'right'), 5),
 (('infection', '<\\s>'), 5),
 (('significant', 'change'), 5),
 (('there', 'is'), 5),
 (('is', 'detected'), 4),
 (('there', 'are'), 4),
 (('old', 'tb'), 4),
 (('tb', 'or'), 4),
 (('no', 'significant'), 4),
 (('could', 'be'), 4),
 (('inflitration', 'is'), 3),
 (('detected', '<\\s>'), 3),
 (('the', 'rul'), 3),
 (('and', 'rll'), 3),
 (('region', '<\\s>'), 3)]

In [999]:
v1_trigramk = createPatternNgrams(v1_report, 3, v1_keyword_3)
list(v1_trigramk.items())[:20]

[(('<s>', 'there', 'is'), 5),
 (('<s>', 'there', 'are'), 4),
 (('no', 'significant', 'change'), 4),
 (('inflitration', 'is', 'detected'), 3),
 (('is', 'detected', '<\\s>'), 3),
 (('old', 'tb', 'or'), 3),
 (('at', 'the', 'both'), 3),
 (('inflamatory', 'process', '<\\s>'), 3),
 (('<s>', 'minimal', 'rul'), 2),
 (('minimal', 'rul', 'interstitial'), 2),
 (('rul', 'interstitial', 'inflitration'), 2),
 (('interstitial', 'inflitration', 'is'), 2),
 (('at', 'the', 'rul'), 2),
 (('two', 'illed', 'define'), 2),
 (('illed', 'define', 'soft'), 2),
 (('define', 'soft', 'tissue'), 2),
 (('soft', 'tissue', 'densty'), 2),
 (('tissue', 'densty', 'lesions'), 2),
 (('densty', 'lesions', 'at'), 2),
 (('lesions', 'at', 'right'), 2)]

In [1000]:
v1_fourgramk = createPatternNgrams(v1_report, 4, v1_keyword_3)
list(v1_fourgramk.items())[:20]

[(('<s>', 'minimal', 'rul', 'interstitial'), 2),
 (('minimal', 'rul', 'interstitial', 'inflitration'), 2),
 (('rul', 'interstitial', 'inflitration', 'is'), 2),
 (('interstitial', 'inflitration', 'is', 'detected'), 2),
 (('inflitration', 'is', 'detected', '<\\s>'), 2),
 (('two', 'illed', 'define', 'soft'), 2),
 (('illed', 'define', 'soft', 'tissue'), 2),
 (('define', 'soft', 'tissue', 'densty'), 2),
 (('soft', 'tissue', 'densty', 'lesions'), 2),
 (('tissue', 'densty', 'lesions', 'at'), 2),
 (('densty', 'lesions', 'at', 'right'), 2),
 (('lesions', 'at', 'right', 'hilar'), 2),
 (('at', 'right', 'hilar', 'region'), 2),
 (('right', 'hilar', 'region', 'and'), 2),
 (('hilar', 'region', 'and', 'left'), 2),
 (('region', 'and', 'left', 'heart'), 2),
 (('and', 'left', 'heart', 'border'), 2),
 (('left', 'heart', 'border', 'size'), 2),
 (('<s>', 'there', 'are', 'cardiomegaly'), 2),
 (('there', 'are', 'cardiomegaly', 'with'), 2)]

## Difference between Result 1 and 2

In [1001]:
findDiffData('Inspectra Lung Opacity v1', v1_data_old, v1_data)

1.0 -1.0 ['<s> chest pa upright lat <\\s>', '<s> known case odl ptb with lul mass compared to film on 4 4 2005 <\\s>', '<s> no signficiatn changed of mass at lul with surrouding interstitial infinltration <\\s>', '<s> and reticular infiltration at rul <\\s>', '<s> no gross adjacent rib destruction is evident <\\s>', '<s> no new lung lesion is seen <\\s>', '<s> no pleural effusion is shown <\\s>', '<s> heart is normal in size <\\s>', '<s> tortuous calcific thoracic aorta is seen <\\s>', '<s> ds <\\s>']
nan -1.0 ['<s> cxr pa upright <\\s>', '<s> minimal fibroinfitlration at rll <\\s>', '<s> fibrosis rul <\\s>', '<s> prominent of both hilar regions are seen which may be lymphadenopathy or prominent pulmonary trunk <\\s>', '<s> normal cardiothoracic ratio <\\s>', '<s> artherosclerotic changes of aorta <\\s>', '<s> bony thorax is intact <\\s>', '<s> right pleural effusion is also suspected <\\s>', '<s> please clinical correlation <\\s>']
nan 1.0 ['<s> chest pa upright <\\s>', '<s> diffuse m

# Edema